###### 필요 라이브러리 가져오기

In [1]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
import tensorflow as tf

###### 모델 하이퍼 파라미터 셋팅

In [2]:
# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 100
data_augmentation = False
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2
    
# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

###### 학습 데이터(CIFAR-10) 가져오기 및 전처리

In [3]:
# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

'''
# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean
'''

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


###### 학습률(learning rate) 조정 함수 정의

In [4]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

###### ResNet-20 모델 설계

In [5]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]
    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

###### 모델 컴파일

In [6]:
if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model_v1 = resnet_v1(input_shape=input_shape, depth=depth)

model_v1.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model_v1.summary()
print(model_type)

Learning rate:  0.001
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization[0][0]        
________________________________________________________________________

C:\Users\SQUARE\AppData\Roaming\Python\Python37\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


###### 모델 학습

In [7]:
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model_v1.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2,
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model_v1.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4)


Not using data augmentation.
Epoch 1/100
1250/1250 [==============================] - 34s 23ms/step - loss: 1.5909 - accuracy: 0.4815 - val_loss: 1.8299 - val_accuracy: 0.4602
Epoch 2/100
1250/1250 [==============================] - 28s 22ms/step - loss: 1.1685 - accuracy: 0.6445 - val_loss: 1.3701 - val_accuracy: 0.5886
Epoch 3/100
1250/1250 [==============================] - 28s 22ms/step - loss: 0.9834 - accuracy: 0.7104 - val_loss: 1.0231 - val_accuracy: 0.6951
Epoch 4/100
1250/1250 [==============================] - 28s 22ms/step - loss: 0.8728 - accuracy: 0.7537 - val_loss: 1.1733 - val_accuracy: 0.6669
Epoch 5/100
1250/1250 [==============================] - 28s 22ms/step - loss: 0.7906 - accuracy: 0.7856 - val_loss: 1.4491 - val_accuracy: 0.5988
Epoch 6/100
1250/1250 [==============================] - 28s 23ms/step - loss: 0.7366 - accuracy: 0.8065 - val_loss: 1.0196 - val_accuracy: 0.7162
Epoch 7/100
1250/1250 [==============================] - 28s 22ms/step - loss: 0.6806 - a

###### 모델 평가 (using test data)

In [5]:
# Score trained model.
scores = model_v1.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 5s 9ms/step - loss: 1.3573 - accuracy: 0.7650
Test loss: 1.3572852611541748
Test accuracy: 0.7649999856948853


 ###### Model Save

In [42]:
from keras.models import load_model
model_v1.save('models/cifar10_model-validation.h5')

C:\Users\SQUARE\AppData\Roaming\Python\Python37\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


###### Model Load

In [4]:
from keras.models import load_model
model_v1 = load_model('models/cifar10_model-validation.h5')

###### Extract activation values

###### 레이블 별 데이터 추출

In [6]:
airplane_label = [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
automobile_label = [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]
bird_label = [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]
cat_label = [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]
deer_label = [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]
dog_label = [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]
frog_label = [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]
horse_label = [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]
ship_label = [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]
truck_label = [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]

x_airplane = []
x_automobile = []
x_bird = []
x_cat = []
x_deer = []
x_dog = []
x_frog = []
x_horse = []
x_ship = []
x_truck = []

for idx in range(50000):
    if np.array_equal(y_train[idx], airplane_label):
       x_airplane.append(x_train[idx])
    elif np.array_equal(y_train[idx], automobile_label):
       x_automobile.append(x_train[idx])
    elif np.array_equal(y_train[idx], bird_label):
       x_bird.append(x_train[idx])
    elif np.array_equal(y_train[idx], cat_label):
       x_cat.append(x_train[idx])
    elif np.array_equal(y_train[idx], deer_label):
       x_deer.append(x_train[idx])
    elif np.array_equal(y_train[idx], dog_label):
       x_dog.append(x_train[idx])
    elif np.array_equal(y_train[idx], frog_label):
       x_frog.append(x_train[idx])
    elif np.array_equal(y_train[idx], horse_label):
       x_horse.append(x_train[idx])
    elif np.array_equal(y_train[idx], ship_label):
       x_ship.append(x_train[idx])
    elif np.array_equal(y_train[idx], truck_label):
       x_truck.append(x_train[idx])

x_airplane = np.array(x_airplane)
x_automobile = np.array(x_automobile)
x_bird = np.array(x_bird)
x_cat = np.array(x_cat)
x_deer = np.array(x_deer)
x_dog = np.array(x_dog)
x_frog = np.array(x_frog)
x_horse = np.array(x_horse)
x_ship = np.array(x_ship)
x_truck = np.array(x_truck)

In [7]:
print('x_airplane', x_airplane.shape, 'x_automobile', x_automobile.shape, 'x_bird', x_bird.shape,
     'x_cat', x_cat.shape, 'x_deer', x_deer.shape, 'x_dog', x_dog.shape, 'x_frog', x_frog.shape,
     'x_horse', x_horse.shape, 'x_ship', x_ship.shape, 'x_truck', x_truck.shape)

x_airplane (5000, 32, 32, 3) x_automobile (5000, 32, 32, 3) x_bird (5000, 32, 32, 3) x_cat (5000, 32, 32, 3) x_deer (5000, 32, 32, 3) x_dog (5000, 32, 32, 3) x_frog (5000, 32, 32, 3) x_horse (5000, 32, 32, 3) x_ship (5000, 32, 32, 3) x_truck (5000, 32, 32, 3)


###### 데이터 별 activation value 추출

In [8]:
#layer_indices = [3, 6, 10, 13, 17, 20, 24, 27, 32, 35, 39, 42, 46, 49, 54, 57, 61, 64, 68]
#layer_indices = [27, 32, 35, 39, 42, 46, 49, 54, 57, 61, 64, 68]
layer_indices = [49, 54, 57, 61, 64, 68]
k_function = K.function([model_v1.layers[0].input], [model_v1.layers[49].output, model_v1.layers[54].output, model_v1.layers[57].output, 
                                                            model_v1.layers[61].output, model_v1.layers[64].output, model_v1.layers[68].output])

def get_layer_output(dataSet, _range):
    layer_outputs = []
    
    for data_idx in _range:
        start_t = time.time()
        layer_output_list = []

        '''
        for i in layer_indices:
            k_function = K.function([model_v1.layers[0].input], [model_v1.layers[i].output])

            data_ip = np.reshape(dataSet[data_idx], (1, 32, 32, 3))
            layer_output_list.append(k_function([data_ip]))
        '''
                
        data_ip = np.reshape(dataSet[data_idx], (1, 32, 32, 3))
        a, b, c, d, e, f = k_function([data_ip])
        
        layer_output_list.extend([a, b, c, d, e, f])
        layer_outputs.append(layer_output_list)
        
        end_t = time.time()
#         print(data_idx, str(end_t - start_t) + 'sec')
        
    return layer_outputs

###### convet List of layer output to Numpy Array

In [9]:
def act_value_list_to_np_array(value_list):
    act_arr_list = []
    
    for each in value_list:
        act_arraies = []
        np_layer_out1 = np.array(each[0])
        for idx in range(1, 6):
            np_layer_out1 = np.concatenate((np_layer_out1, each[idx]), axis=0)

        #0이 아닌 activation value는 모두 1로 변경
        np_layer_out1[np_layer_out1 > 0] = 1
        '''np_layer_out1[np_layer_out1 >= 1] = 1
        np_layer_out1[np_layer_out1 < 1] = 0'''

        act_arraies.append(np_layer_out1)
        act_arr_list.append(act_arraies)

    return act_arr_list

**자료구조 형태**
<br>(list, list, (np.array))
<br>~~(0:500, 0, 0:6, 8, 8, 64)~~
<br>(0:500, 1, (6, 8, 8, 64))

###### 500개의 데이터를 합산

In [10]:
def combine_data_list(data_list, number):
    combined_list = []

    arr1 = np.zeros(shape=(6, 8, 8, 64))

    for idx1 in range(number):
        arr1 += data_list[idx1][0]
        
    combined_list.append(arr1)
    
    return combined_list

##### 일정 threshold 이상인 neuron의 값만 추출

In [11]:
def get_activated_neurons(data_list, threshold):
    for idx in range(0, 1):
        data_list[idx][data_list[idx] < threshold] = 0
        data_list[idx][data_list[idx] >= threshold] = 1
        
    return data_list

#### get activated neurons from data

In [12]:
def get_activated_neurons_from_data(data, _range, number, threshold):
    data_layer_output = get_layer_output(data, _range)
#     print('get_layer_output is done. ', len(data_layer_output))
    
    data_act_arr_list = act_value_list_to_np_array(data_layer_output)
#     print('act_value_list_to_np_array is done. ', len(data_act_arr_list))
        
    combined_data_list = combine_data_list(data_act_arr_list, number)
#     print('combine_data_list is done.')

    combined_data_list = get_activated_neurons(combined_data_list, threshold)
#     print('get_activated_neurons is done.')

    return combined_data_list

In [13]:
combined_airplane_list = get_activated_neurons_from_data(x_airplane, range(0, 500), 500, 250)
print('airplane is done.')
combined_automobile_list = get_activated_neurons_from_data(x_automobile, range(0, 500), 500, 250)
print('automobile is done.')
combined_bird_list = get_activated_neurons_from_data(x_bird, range(0, 500), 500, 250)
print('bird is done.')
combined_cat_list = get_activated_neurons_from_data(x_cat, range(0, 500), 500, 250)
print('cat is done.')
combined_deer_list = get_activated_neurons_from_data(x_deer, range(0, 500), 500, 250)
print('deer is done.')
combined_dog_list = get_activated_neurons_from_data(x_dog, range(0, 500), 500, 250)
print('dog is done.')
combined_frog_list = get_activated_neurons_from_data(x_frog, range(0, 500), 500, 250)
print('frog is done.')
combined_horse_list = get_activated_neurons_from_data(x_horse, range(0, 500), 500, 250)
print('horse is done.')
combined_ship_list = get_activated_neurons_from_data(x_ship, range(0, 500), 500, 250)
print('ship is done.')
combined_truck_list = get_activated_neurons_from_data(x_truck, range(0, 500), 500, 250)
print('truck is done.')

airplane is done.
automobile is done.
bird is done.
cat is done.
deer is done.
dog is done.
frog is done.
horse is done.
ship is done.
truck is done.


In [14]:
print('airplane', np.count_nonzero(combined_airplane_list[0]))
print('automobile', np.count_nonzero(combined_automobile_list[0]))
print('bird', np.count_nonzero(combined_bird_list[0]))
print('cat', np.count_nonzero(combined_cat_list[0]))
print('deer', np.count_nonzero(combined_deer_list[0]))
print('dog', np.count_nonzero(combined_dog_list[0]))
print('frog', np.count_nonzero(combined_frog_list[0]))
print('horse', np.count_nonzero(combined_horse_list[0]))
print('ship', np.count_nonzero(combined_ship_list[0]))
print('truck', np.count_nonzero(combined_truck_list[0]))

airplane 4148
automobile 4333
bird 3978
cat 3991
deer 4169
dog 4097
frog 4224
horse 4094
ship 4262
truck 4419


In [19]:
print(combined_airplane_list[0].shape)
print(combined_automobile_list[0].size)
print(combined_bird_list[0].size)
print(combined_cat_list[0].size)
print(combined_deer_list[0].size)
print(combined_dog_list[0].size)
print(combined_frog_list[0].size)
print(combined_horse_list[0].size)
print(combined_ship_list[0].size)
print(combined_truck_list[0].size)

(6, 8, 8, 64)
24576
24576
24576
24576
24576
24576
24576
24576
24576


###### Jaccard Similarity 계산

In [14]:
def get_union(first, second):
    count = 0
    
    for idx in range(1):
        count += np.count_nonzero(first[idx] + second[idx])

    return count

def get_intersact(first, second):
    group1 = np.zeros(shape=(6,  8, 8, 64))
    
    group1 = first[0] + second[0]
    
    group1[group1 < 2] = 0
    
    return np.count_nonzero(group1)
    
def get_jaccard_similarity(first, second):
    union = get_union(first, second)
    intersact = get_intersact(first, second)
    
    #print('union ', union)
    #print('intersact ', intersact)
    
    return intersact / union

### RQ 1. 유사 클래스 similarity 확인

### RQ 2. 식별 기준 측정 (training ↔ test)

###### activation value 추출 for test(unseen) data

In [15]:
tx_airplane = []
tx_automobile = []
tx_bird = []
tx_cat = []
tx_deer = []
tx_dog = []
tx_frog = []
tx_horse = []
tx_ship = []
tx_truck = []

for idx in range(10000):
    if np.array_equal(y_test[idx], airplane_label):
       tx_airplane.append(x_test[idx])
    elif np.array_equal(y_test[idx], automobile_label):
       tx_automobile.append(x_test[idx])
    elif np.array_equal(y_test[idx], bird_label):
       tx_bird.append(x_test[idx])
    elif np.array_equal(y_test[idx], cat_label):
       tx_cat.append(x_test[idx])
    elif np.array_equal(y_test[idx], deer_label):
       tx_deer.append(x_test[idx])
    elif np.array_equal(y_test[idx], dog_label):
       tx_dog.append(x_test[idx])
    elif np.array_equal(y_test[idx], frog_label):
       tx_frog.append(x_test[idx])
    elif np.array_equal(y_test[idx], horse_label):
       tx_horse.append(x_test[idx])
    elif np.array_equal(y_test[idx], ship_label):
       tx_ship.append(x_test[idx])
    elif np.array_equal(y_test[idx], truck_label):
       tx_truck.append(x_test[idx])

tx_airplane = np.array(tx_airplane)
tx_automobile = np.array(tx_automobile)
tx_bird = np.array(tx_bird)
tx_cat = np.array(tx_cat)
tx_deer = np.array(tx_deer)
tx_dog = np.array(tx_dog)
tx_frog = np.array(tx_frog)
tx_horse = np.array(tx_horse)
tx_ship = np.array(tx_ship)
tx_truck = np.array(tx_truck)

In [16]:
print('tx_airplane', tx_airplane.shape, 'tx_automobile', tx_automobile.shape, 'tx_bird', tx_bird.shape,
     'tx_cat', tx_cat.shape, 'tx_deer', tx_deer.shape, 'tx_dog', tx_dog.shape, 'tx_frog', tx_frog.shape,
     'tx_horse', tx_horse.shape, 'tx_ship', tx_ship.shape, 'tx_truck', tx_truck.shape)

tx_airplane (1000, 32, 32, 3) tx_automobile (1000, 32, 32, 3) tx_bird (1000, 32, 32, 3) tx_cat (1000, 32, 32, 3) tx_deer (1000, 32, 32, 3) tx_dog (1000, 32, 32, 3) tx_frog (1000, 32, 32, 3) tx_horse (1000, 32, 32, 3) tx_ship (1000, 32, 32, 3) tx_truck (1000, 32, 32, 3)


In [19]:
'''t_combined_airplane_list = get_activated_neurons_from_data(tx_airplane, range(0, 100), 100, 50)
print('airplane is done.')
t_combined_automobile_list = get_activated_neurons_from_data(tx_automobile, range(0, 100), 100, 50)
print('automobile is done.')
t_combined_bird_list = get_activated_neurons_from_data(tx_bird, range(0, 100), 100, 50)
print('bird is done.')
t_combined_cat_list = get_activated_neurons_from_data(tx_cat, range(0, 100), 100, 50)
print('cat is done.')
t_combined_deer_list = get_activated_neurons_from_data(tx_deer, range(0, 100), 100, 50)
print('deer is done.')
t_combined_dog_list = get_activated_neurons_from_data(tx_dog, range(0, 100), 100, 50)
print('dog is done.')
t_combined_frog_list = get_activated_neurons_from_data(tx_frog, range(0, 100), 100, 50)
print('frog is done.')
t_combined_horse_list = get_activated_neurons_from_data(tx_horse, range(0, 100), 100, 50)
print('horse is done.')
t_combined_ship_list = get_activated_neurons_from_data(tx_ship, range(0, 100), 100, 50)
print('ship is done.')
t_combined_truck_list = get_activated_neurons_from_data(tx_truck, range(0, 100), 100, 50)
print('truck is done.')'''

0 0.015209436416625977sec
1 0.03125190734863281sec
2 0.01562643051147461sec
3 0.03125262260437012sec
4 0.015625715255737305sec
5 0.01562643051147461sec
6 0.03125190734863281sec
7 0.015626192092895508sec
8 0.03125143051147461sec
9 0.01562643051147461sec
10 0.015626907348632812sec
11 0.031250715255737305sec
12 0.01562786102294922sec
13 0.03125190734863281sec
14 0.015624761581420898sec
15 0.015625953674316406sec
16 0.03125286102294922sec
17 0.015625953674316406sec
18 0.03125190734863281sec
19 0.01562643051147461sec
20 0.015626192092895508sec
21 0.03125143051147461sec
22 0.01562666893005371sec
23 0.015625953674316406sec
24 0.03125309944152832sec
25 0.015625476837158203sec
26 0.031252145767211914sec
27 0.015625953674316406sec
28 0.01562643051147461sec
29 0.03125190734863281sec
30 0.015626192092895508sec
31 0.031252145767211914sec
32 0.015626192092895508sec
33 0.015625953674316406sec
34 0.031252384185791016sec
35 0.015625953674316406sec
36 0.015625953674316406sec
37 0.031252384185791016sec
3

7 0.01562666893005371sec
8 0.03125143051147461sec
9 0.015626192092895508sec
10 0.03125190734863281sec
11 0.015626192092895508sec
12 0.015625953674316406sec
13 0.031252384185791016sec
14 0.015626192092895508sec
15 0.03125190734863281sec
16 0.015626192092895508sec
17 0.01562643051147461sec
18 0.031252145767211914sec
19 0.015625953674316406sec
20 0.015625953674316406sec
21 0.031252384185791016sec
22 0.015625953674316406sec
23 0.031252145767211914sec
24 0.015626192092895508sec
25 0.015625953674316406sec
26 0.031252384185791016sec
27 0.015626192092895508sec
28 0.015625715255737305sec
29 0.031252145767211914sec
30 0.015625953674316406sec
31 0.031252384185791016sec
32 0.015625953674316406sec
33 0.01562643051147461sec
34 0.03125190734863281sec
35 0.01562643051147461sec
36 0.03125166893005371sec
37 0.015625715255737305sec
38 0.01562666893005371sec
39 0.031252145767211914sec
40 0.01562643051147461sec
41 0.015626192092895508sec
42 0.03125166893005371sec
43 0.01562643051147461sec
44 0.031252145767

5 0.015624761581420898sec
6 0.03125166893005371sec
7 0.015625953674316406sec
8 0.03125190734863281sec
9 0.015626192092895508sec
10 0.015626192092895508sec
11 0.031252384185791016sec
12 0.015625953674316406sec
13 0.03125190734863281sec
14 0.01562666893005371sec
15 0.015625715255737305sec
16 0.031252145767211914sec
17 0.015625953674316406sec
18 0.03125262260437012sec
19 0.015625953674316406sec
20 0.015625953674316406sec
21 0.031252145767211914sec
22 0.015626192092895508sec
23 0.031252384185791016sec
24 0.01562643051147461sec
25 0.0156252384185791sec
26 0.031252384185791016sec
27 0.015625953674316406sec
28 0.031252384185791016sec
29 0.015625953674316406sec
30 0.015626192092895508sec
31 0.031252145767211914sec
32 0.01562643051147461sec
33 0.03125166893005371sec
34 0.015626192092895508sec
35 0.015626192092895508sec
36 0.031252145767211914sec
37 0.01562643051147461sec
38 0.03125190734863281sec
39 0.015626192092895508sec
40 0.015625953674316406sec
41 0.03125143051147461sec
42 0.01562595367431

3 0.01562643051147461sec
4 0.03125119209289551sec
5 0.015625953674316406sec
6 0.03125190734863281sec
7 0.015626192092895508sec
8 0.015626192092895508sec
9 0.031252384185791016sec
10 0.015625476837158203sec
11 0.03125262260437012sec
12 0.01562643051147461sec
13 0.015625953674316406sec
14 0.03125190734863281sec
15 0.015625953674316406sec
16 0.031252145767211914sec
17 0.01562643051147461sec
18 0.015625953674316406sec
19 0.031252145767211914sec
20 0.015625476837158203sec
21 0.03125286102294922sec
22 0.015626192092895508sec
23 0.015626192092895508sec
24 0.03125190734863281sec
25 0.015626192092895508sec
26 0.03125190734863281sec
27 0.01562643051147461sec
28 0.015626192092895508sec
29 0.03125166893005371sec
30 0.015626192092895508sec
31 0.031252145767211914sec
32 0.015626192092895508sec
33 0.01562643051147461sec
34 0.03125190734863281sec
35 0.015626192092895508sec
36 0.031252145767211914sec
37 0.015625953674316406sec
38 0.015626192092895508sec
39 0.031252145767211914sec
40 0.01562619209289550

In [20]:
'''print('airplane', np.count_nonzero(t_combined_airplane_list[0]))
print('automobile', np.count_nonzero(t_combined_automobile_list[0]))
print('bird', np.count_nonzero(t_combined_bird_list[0]))
print('cat', np.count_nonzero(t_combined_cat_list[0]))
print('deer', np.count_nonzero(t_combined_deer_list[0]))
print('dog', np.count_nonzero(t_combined_dog_list[0]))
print('frog', np.count_nonzero(t_combined_frog_list[0]))
print('horse', np.count_nonzero(t_combined_horse_list[0]))
print('ship', np.count_nonzero(t_combined_ship_list[0]))
print('truck', np.count_nonzero(t_combined_truck_list[0]))'''

airplane 4083
automobile 4392
bird 3910
cat 4064
deer 4127
dog 4136
frog 4275
horse 4085
ship 4278
truck 4275


In [20]:
'''air_air_similarity = get_jaccard_similarity(combined_airplane_list, t_combined_airplane_list)
auto_auto_similarity = get_jaccard_similarity(combined_automobile_list, t_combined_automobile_list)
bird_bird_similarity = get_jaccard_similarity(combined_bird_list, t_combined_bird_list)
cat_cat_similarity = get_jaccard_similarity(combined_cat_list, t_combined_cat_list)
deer_deer_similarity = get_jaccard_similarity(combined_deer_list, t_combined_deer_list)
dog_dog_similarity = get_jaccard_similarity(combined_dog_list, t_combined_dog_list)
frog_frog_similarity = get_jaccard_similarity(combined_frog_list, t_combined_frog_list)
horse_horse_similarity = get_jaccard_similarity(combined_horse_list, t_combined_horse_list)
ship_ship_similarity = get_jaccard_similarity(combined_ship_list, t_combined_ship_list)
truck_truck_similarity = get_jaccard_similarity(combined_truck_list, t_combined_truck_list)

print('airplane↔t_airplane', air_air_similarity)
print('automobile↔t_automobile', auto_auto_similarity)
print('bird↔t_bird', bird_bird_similarity)
print('cat↔t_cat', cat_cat_similarity)
print('deer↔t_deer', deer_deer_similarity)
print('dog↔t_dog', dog_dog_similarity)
print('frog↔t_frog', frog_frog_similarity)
print('horse↔t_horse', horse_horse_similarity)
print('ship↔t_ship', ship_ship_similarity)
print('truck↔t_truck', truck_truck_similarity)'''

airplane↔t_airplane 0.8447404685346808
automobile↔t_automobile 0.8412428176207704
bird↔t_bird 0.8385093167701864
cat↔t_cat 0.835361653272101
deer↔t_deer 0.8493400091033227
dog↔t_dog 0.8367531003382187
frog↔t_frog 0.8471028860982596
horse↔t_horse 0.8434065934065934
ship↔t_ship 0.8567354102676612
truck↔t_truck 0.8539969169786391


### RQ 3. 데이터 클래스 식별

###### 하나의 클래스 데이터를 입력했을 때, 활성화 되는 뉴런 중 상위 α%를 뉴런 클러스터로 식별

In [32]:
print(np.shape(combined_truck_list))

(1, 6, 8, 8, 64)


In [17]:
def get_nth_value(data, nth):
    flat_data = np.reshape(data, (1, np.product(data.shape)))[0]
    #print('flat_data', flat_data.shape)
    #print(flat_data)
    
    flat_data = np.sort(flat_data)[::-1]
    #print('flat_data', flat_data.shape)
    #print(flat_data)
    
    return flat_data[nth]

In [44]:
def get_neuron_cluster_with_one_data(data):

    layer_output_list = []

    data_ip = np.reshape(data, (1, 32, 32, 3))
    a, b, c, d, e, f = k_function([data_ip])
    layer_output_list.extend([a, b, c, d, e, f]) # (6, 1, 8, 8, 64)    
    
    # convert shape from (6, 1, 8, 8, 64) to (1, 6, 8, 8, 64)
    np_layer_output = np.array(layer_output_list[0])
    for idx in range(1, 6):
        np_layer_output = np.concatenate((np_layer_output, layer_output_list[idx]), axis=0)
    
    converted_layer_output_list = []
    converted_layer_output_list.append(np_layer_output)
    
    # 1개의 데이터에 대한 threshold 계산
    nonzero = np.count_nonzero(np_layer_output)
    nth = round(nonzero * 0.5)
#     print('nth=', nth)
    
    # identify neuron cluster
    threshold = get_nth_value(np_layer_output, nth)
    
    converted_layer_output_list[0][converted_layer_output_list[0] < threshold] = 0
    converted_layer_output_list[0][converted_layer_output_list[0] >= threshold] = 1
    
    
    return converted_layer_output_list

In [83]:
t_one_airplane_list = get_neuron_cluster_with_one_data(tx_airplane[0])
t_one_automobile_list = get_neuron_cluster_with_one_data(tx_automobile[0])
t_one_bird_list = get_neuron_cluster_with_one_data(tx_bird[0])
t_one_cat_list = get_neuron_cluster_with_one_data(tx_cat[0])
t_one_deer_list = get_neuron_cluster_with_one_data(tx_deer[0])
t_one_dog_list = get_neuron_cluster_with_one_data(tx_dog[0])
t_one_frog_list = get_neuron_cluster_with_one_data(tx_frog[0])
t_one_horse_list = get_neuron_cluster_with_one_data(tx_horse[0])
t_one_ship_list = get_neuron_cluster_with_one_data(tx_ship[0])
t_one_truck_list = get_neuron_cluster_with_one_data(tx_truck[0])


nth= 3458
nth= 3540
nth= 3436
nth= 3512
nth= 3180
nth= 3498
nth= 3426
nth= 3488
nth= 3483
nth= 3614


In [17]:
air_np_label = np.array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

# Score trained model.
scores = model_v1.predict(x_test)

In [19]:
airplane_label = [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
automobile_label = [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]
bird_label = [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]
cat_label = [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]
deer_label = [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]
dog_label = [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]
frog_label = [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]
horse_label = [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]
ship_label = [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]
truck_label = [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]

def get_int_label(label):
    if np.array_equal(label, airplane_label):
        return 0
    elif np.array_equal(label, automobile_label):
        return 1
    elif np.array_equal(label, bird_label):
        return 2
    elif np.array_equal(label, cat_label):
        return 3
    elif np.array_equal(label, deer_label):
        return 4
    elif np.array_equal(label, dog_label):
        return 5
    elif np.array_equal(label, frog_label):
        return 6
    elif np.array_equal(label, horse_label):
        return 7
    elif np.array_equal(label, ship_label):
        return 8
    elif np.array_equal(label, truck_label):
        return 9

###### 여기서부터 label없이 테스트 데이터 선정하기

In [22]:
CNC_List = [combined_airplane_list, combined_automobile_list, combined_bird_list, combined_cat_list, combined_deer_list,
           combined_dog_list, combined_frog_list, combined_horse_list, combined_ship_list, combined_truck_list]

def get_top2_similarities(combined_data_list):
    similarity_list = []
    
    for idx in range(10):
        similarity_list.append(get_jaccard_similarity(CNC_List[idx], combined_data_list))
        
    similarity_list.sort(reverse=True)
    
    return similarity_list[0], similarity_list[1]

In [42]:
def selectTestDataWithoutLabel(origin_x_data, origin_y_data):
    diff_x_test = []
    diff_y_test = []

    data_size = len(origin_x_data)
    for idx in range(data_size):
        x_test_neuron_cluster = get_neuron_cluster_with_one_data(origin_x_data[idx])
        top1_sim, top2_sim = get_top2_similarities(x_test_neuron_cluster)
        
        diff = top1_sim * 0.05
        if top1_sim < (top2_sim + diff):
            diff_x_test.append(origin_x_data[idx])
            diff_y_test.append(origin_y_data[idx])
        
    np_diff_x_test = np.array(diff_x_test)
    np_diff_y_test = np.array(diff_y_test)
    
    return np_diff_x_test, np_diff_y_test

In [71]:
index_range = range(0, 10000)

In [72]:
start_t = time.time()

selected_x_test, selected_y_test = selectTestDataWithoutLabel(x_test[index_range], y_test[index_range])

end_t = time.time()
print(str(end_t - start_t) + 'sec')

print(selected_x_test.shape)
print(selected_y_test.shape)

256.2910215854645sec
(4201, 32, 32, 3)
(4201, 10)


In [73]:
sel_scores = model_v1.evaluate(selected_x_test, selected_y_test, verbose=1)
print('Test loss:', sel_scores[0])
print('Test accuracy:', sel_scores[1])

132/132 [==============================] - 2s 12ms/step - loss: 2.0725 - accuracy: 0.5708
Test loss: 2.0724987983703613
Test accuracy: 0.5708164572715759


In [19]:
CNC_List = [combined_airplane_list, combined_automobile_list, combined_bird_list, combined_cat_list, combined_deer_list,
           combined_dog_list, combined_frog_list, combined_horse_list, combined_ship_list, combined_truck_list]

'''similarities = [air_air_similarity, auto_auto_similarity, bird_bird_similarity, cat_cat_similarity, deer_deer_similarity, dog_dog_similarity,
               frog_frog_similarity, horse_horse_similarity, ship_ship_similarity, truck_truck_similarity]'''

def get_max_class_label(combined_data_list):
    max_sim = 0
    label = -1
    
    for idx in range(10):
        local_sim = get_jaccard_similarity(CNC_List[idx], combined_data_list)
        #criterion = similarities[idx] * 1.0 # 데이터 셋마다 오차를 고려해 90%로 설정 → 현재는 100%로 사용, 큰 차이 없음
        #print('crit = ', criterion, ', sim = ', local_sim)
        if local_sim > max_sim:
            max_sim = local_sim
            label = idx
        
    return max_sim, label

In [29]:
count = 0
wrong_idx = []

for idx in range(0, 10000):
    if np.argmax(scores[idx]) != get_int_label(y_test[idx]):
        print('idx', idx)
        print('expected', np.argmax(scores[idx]))
        print('actual', get_int_label(y_test[idx]))
        count = count + 1
        wrong_idx.append(idx)
    
print(count)

idx 20
expected 4
actual 7
idx 24
expected 4
actual 5
idx 25
expected 4
actual 2
idx 35
expected 8
actual 2
idx 42
expected 4
actual 5
idx 47
expected 7
actual 9
idx 52
expected 7
actual 0
idx 53
expected 2
actual 3
idx 56
expected 5
actual 7
idx 57
expected 5
actual 7
idx 58
expected 7
actual 4
idx 59
expected 4
actual 6
idx 61
expected 5
actual 3
idx 62
expected 4
actual 6
idx 67
expected 0
actual 2
idx 68
expected 5
actual 3
idx 69
expected 4
actual 7
idx 70
expected 6
actual 2
idx 74
expected 9
actual 0
idx 78
expected 5
actual 3
idx 83
expected 4
actual 7
idx 84
expected 5
actual 2
idx 86
expected 5
actual 2
idx 91
expected 6
actual 3
idx 96
expected 4
actual 6
idx 118
expected 6
actual 2
idx 128
expected 6
actual 5
idx 134
expected 6
actual 1
idx 138
expected 5
actual 2
idx 139
expected 0
actual 9
idx 143
expected 5
actual 3
idx 147
expected 6
actual 2
idx 164
expected 2
actual 8
idx 167
expected 6
actual 4
idx 169
expected 4
actual 0
idx 176
expected 4
actual 3
idx 183
expected 

idx 1921
expected 3
actual 2
idx 1924
expected 0
actual 9
idx 1933
expected 5
actual 3
idx 1934
expected 0
actual 7
idx 1935
expected 5
actual 6
idx 1936
expected 7
actual 3
idx 1937
expected 5
actual 1
idx 1938
expected 0
actual 8
idx 1939
expected 7
actual 6
idx 1942
expected 4
actual 2
idx 1944
expected 6
actual 4
idx 1947
expected 5
actual 3
idx 1954
expected 4
actual 2
idx 1957
expected 7
actual 3
idx 1958
expected 4
actual 6
idx 1965
expected 9
actual 0
idx 1969
expected 9
actual 1
idx 1971
expected 3
actual 4
idx 1975
expected 4
actual 6
idx 1983
expected 6
actual 8
idx 1984
expected 4
actual 7
idx 1989
expected 3
actual 6
idx 1993
expected 4
actual 3
idx 1994
expected 9
actual 8
idx 1996
expected 7
actual 3
idx 2005
expected 3
actual 5
idx 2010
expected 5
actual 7
idx 2013
expected 9
actual 6
idx 2016
expected 4
actual 6
idx 2018
expected 2
actual 3
idx 2023
expected 3
actual 5
idx 2033
expected 5
actual 2
idx 2034
expected 4
actual 3
idx 2035
expected 2
actual 6
idx 2036
expec

idx 3810
expected 5
actual 3
idx 3815
expected 4
actual 2
idx 3816
expected 4
actual 2
idx 3821
expected 1
actual 8
idx 3824
expected 4
actual 2
idx 3827
expected 5
actual 4
idx 3828
expected 5
actual 1
idx 3835
expected 5
actual 3
idx 3863
expected 0
actual 2
idx 3864
expected 6
actual 3
idx 3867
expected 5
actual 3
idx 3872
expected 4
actual 7
idx 3883
expected 2
actual 7
idx 3884
expected 6
actual 3
idx 3887
expected 5
actual 2
idx 3890
expected 4
actual 7
idx 3892
expected 7
actual 5
idx 3898
expected 9
actual 1
idx 3902
expected 1
actual 9
idx 3905
expected 4
actual 2
idx 3906
expected 5
actual 3
idx 3909
expected 2
actual 5
idx 3913
expected 4
actual 7
idx 3917
expected 5
actual 7
idx 3920
expected 5
actual 7
idx 3922
expected 0
actual 2
idx 3923
expected 7
actual 3
idx 3927
expected 6
actual 0
idx 3928
expected 6
actual 3
idx 3943
expected 4
actual 0
idx 3947
expected 4
actual 2
idx 3956
expected 6
actual 3
idx 3957
expected 0
actual 9
idx 3962
expected 7
actual 4
idx 3969
expec

idx 5815
expected 5
actual 3
idx 5817
expected 4
actual 3
idx 5819
expected 7
actual 2
idx 5820
expected 7
actual 5
idx 5825
expected 6
actual 3
idx 5830
expected 4
actual 5
idx 5833
expected 5
actual 3
idx 5835
expected 5
actual 3
idx 5837
expected 3
actual 2
idx 5839
expected 6
actual 2
idx 5840
expected 5
actual 2
idx 5841
expected 4
actual 6
idx 5848
expected 5
actual 4
idx 5855
expected 2
actual 7
idx 5859
expected 5
actual 2
idx 5862
expected 3
actual 2
idx 5869
expected 3
actual 7
idx 5871
expected 5
actual 6
idx 5872
expected 4
actual 7
idx 5876
expected 3
actual 6
idx 5882
expected 9
actual 1
idx 5883
expected 4
actual 6
idx 5886
expected 4
actual 2
idx 5890
expected 7
actual 3
idx 5894
expected 4
actual 7
idx 5900
expected 6
actual 4
idx 5908
expected 4
actual 3
idx 5910
expected 3
actual 2
idx 5924
expected 2
actual 4
idx 5925
expected 2
actual 3
idx 5929
expected 5
actual 7
idx 5934
expected 5
actual 3
idx 5944
expected 6
actual 2
idx 5946
expected 6
actual 8
idx 5957
expec

idx 7769
expected 7
actual 4
idx 7771
expected 5
actual 2
idx 7778
expected 6
actual 0
idx 7783
expected 5
actual 3
idx 7784
expected 4
actual 7
idx 7790
expected 0
actual 8
idx 7792
expected 1
actual 9
idx 7797
expected 6
actual 3
idx 7802
expected 3
actual 2
idx 7813
expected 0
actual 3
idx 7815
expected 8
actual 0
idx 7816
expected 4
actual 0
idx 7821
expected 0
actual 2
idx 7822
expected 4
actual 2
idx 7827
expected 5
actual 3
idx 7829
expected 0
actual 2
idx 7830
expected 0
actual 2
idx 7831
expected 4
actual 2
idx 7839
expected 4
actual 3
idx 7843
expected 9
actual 0
idx 7847
expected 4
actual 7
idx 7849
expected 0
actual 2
idx 7852
expected 7
actual 5
idx 7853
expected 5
actual 3
idx 7858
expected 5
actual 2
idx 7866
expected 3
actual 6
idx 7874
expected 4
actual 6
idx 7875
expected 5
actual 2
idx 7877
expected 1
actual 0
idx 7881
expected 4
actual 3
idx 7884
expected 3
actual 9
idx 7887
expected 7
actual 5
idx 7892
expected 9
actual 1
idx 7899
expected 5
actual 7
idx 7906
expec

actual 2
idx 9705
expected 4
actual 6
idx 9707
expected 7
actual 5
idx 9716
expected 6
actual 7
idx 9717
expected 8
actual 9
idx 9718
expected 0
actual 9
idx 9721
expected 5
actual 6
idx 9722
expected 9
actual 3
idx 9729
expected 5
actual 4
idx 9731
expected 6
actual 2
idx 9734
expected 7
actual 4
idx 9736
expected 0
actual 8
idx 9740
expected 6
actual 3
idx 9743
expected 1
actual 9
idx 9744
expected 9
actual 1
idx 9746
expected 0
actual 2
idx 9749
expected 8
actual 0
idx 9750
expected 4
actual 2
idx 9753
expected 6
actual 3
idx 9755
expected 5
actual 3
idx 9757
expected 4
actual 7
idx 9764
expected 0
actual 2
idx 9767
expected 4
actual 2
idx 9776
expected 4
actual 2
idx 9778
expected 5
actual 3
idx 9780
expected 5
actual 4
idx 9781
expected 5
actual 3
idx 9787
expected 0
actual 8
idx 9789
expected 5
actual 6
idx 9790
expected 0
actual 8
idx 9791
expected 6
actual 9
idx 9792
expected 7
actual 8
idx 9793
expected 3
actual 2
idx 9799
expected 4
actual 7
idx 9801
expected 2
actual 6
idx 9

In [30]:
count = 0
right_idx = []

for idx in range(0, 10000):
    if np.argmax(scores[idx]) == get_int_label(y_test[idx]):
        print('idx', idx)
        print('expected', np.argmax(scores[idx]))
        print('actual', get_int_label(y_test[idx]))
        count = count + 1
        right_idx.append(idx)
    
print(count)

idx 0
expected 3
actual 3
idx 1
expected 8
actual 8
idx 2
expected 8
actual 8
idx 3
expected 0
actual 0
idx 4
expected 6
actual 6
idx 5
expected 6
actual 6
idx 6
expected 1
actual 1
idx 7
expected 6
actual 6
idx 8
expected 3
actual 3
idx 9
expected 1
actual 1
idx 10
expected 0
actual 0
idx 11
expected 9
actual 9
idx 12
expected 5
actual 5
idx 13
expected 7
actual 7
idx 14
expected 9
actual 9
idx 15
expected 8
actual 8
idx 16
expected 5
actual 5
idx 17
expected 7
actual 7
idx 18
expected 8
actual 8
idx 19
expected 6
actual 6
idx 21
expected 0
actual 0
idx 22
expected 4
actual 4
idx 23
expected 9
actual 9
idx 26
expected 4
actual 4
idx 27
expected 0
actual 0
idx 28
expected 9
actual 9
idx 29
expected 6
actual 6
idx 30
expected 6
actual 6
idx 31
expected 5
actual 5
idx 32
expected 4
actual 4
idx 33
expected 5
actual 5
idx 34
expected 9
actual 9
idx 36
expected 4
actual 4
idx 37
expected 1
actual 1
idx 38
expected 9
actual 9
idx 39
expected 5
actual 5
idx 40
expected 4
actual 4
idx 41
expe

actual 2
idx 389
expected 5
actual 5
idx 390
expected 1
actual 1
idx 391
expected 2
actual 2
idx 392
expected 6
actual 6
idx 393
expected 2
actual 2
idx 394
expected 9
actual 9
idx 395
expected 6
actual 6
idx 396
expected 2
actual 2
idx 398
expected 0
actual 0
idx 399
expected 3
actual 3
idx 400
expected 9
actual 9
idx 401
expected 8
actual 8
idx 402
expected 7
actual 7
idx 403
expected 8
actual 8
idx 404
expected 8
actual 8
idx 405
expected 4
actual 4
idx 406
expected 0
actual 0
idx 408
expected 8
actual 8
idx 409
expected 2
actual 2
idx 410
expected 7
actual 7
idx 411
expected 9
actual 9
idx 412
expected 3
actual 3
idx 413
expected 6
actual 6
idx 414
expected 1
actual 1
idx 415
expected 9
actual 9
idx 417
expected 7
actual 7
idx 419
expected 7
actual 7
idx 420
expected 4
actual 4
idx 423
expected 0
actual 0
idx 424
expected 2
actual 2
idx 425
expected 9
actual 9
idx 427
expected 4
actual 4
idx 429
expected 6
actual 6
idx 431
expected 5
actual 5
idx 432
expected 3
actual 3
idx 435
exp

actual 1
idx 782
expected 7
actual 7
idx 783
expected 6
actual 6
idx 784
expected 7
actual 7
idx 785
expected 0
actual 0
idx 786
expected 3
actual 3
idx 787
expected 9
actual 9
idx 788
expected 6
actual 6
idx 789
expected 8
actual 8
idx 791
expected 0
actual 0
idx 794
expected 7
actual 7
idx 795
expected 7
actual 7
idx 798
expected 7
actual 7
idx 800
expected 7
actual 7
idx 801
expected 1
actual 1
idx 803
expected 7
actual 7
idx 804
expected 4
actual 4
idx 805
expected 4
actual 4
idx 807
expected 4
actual 4
idx 808
expected 7
actual 7
idx 809
expected 7
actual 7
idx 812
expected 7
actual 7
idx 813
expected 2
actual 2
idx 814
expected 0
actual 0
idx 815
expected 8
actual 8
idx 816
expected 9
actual 9
idx 817
expected 5
actual 5
idx 818
expected 8
actual 8
idx 820
expected 6
actual 6
idx 822
expected 0
actual 0
idx 823
expected 8
actual 8
idx 824
expected 7
actual 7
idx 825
expected 3
actual 3
idx 826
expected 7
actual 7
idx 827
expected 6
actual 6
idx 830
expected 1
actual 1
idx 832
exp

idx 1208
expected 3
actual 3
idx 1209
expected 7
actual 7
idx 1210
expected 0
actual 0
idx 1211
expected 6
actual 6
idx 1212
expected 1
actual 1
idx 1213
expected 8
actual 8
idx 1214
expected 5
actual 5
idx 1215
expected 7
actual 7
idx 1216
expected 8
actual 8
idx 1217
expected 4
actual 4
idx 1218
expected 8
actual 8
idx 1219
expected 3
actual 3
idx 1221
expected 9
actual 9
idx 1223
expected 8
actual 8
idx 1224
expected 7
actual 7
idx 1225
expected 6
actual 6
idx 1226
expected 6
actual 6
idx 1227
expected 3
actual 3
idx 1229
expected 1
actual 1
idx 1231
expected 9
actual 9
idx 1232
expected 1
actual 1
idx 1233
expected 4
actual 4
idx 1234
expected 1
actual 1
idx 1235
expected 5
actual 5
idx 1237
expected 0
actual 0
idx 1238
expected 1
actual 1
idx 1240
expected 2
actual 2
idx 1241
expected 0
actual 0
idx 1243
expected 8
actual 8
idx 1244
expected 5
actual 5
idx 1245
expected 6
actual 6
idx 1246
expected 7
actual 7
idx 1249
expected 4
actual 4
idx 1250
expected 7
actual 7
idx 1252
expec

idx 1598
expected 5
actual 5
idx 1599
expected 5
actual 5
idx 1600
expected 8
actual 8
idx 1601
expected 9
actual 9
idx 1602
expected 7
actual 7
idx 1603
expected 3
actual 3
idx 1604
expected 1
actual 1
idx 1608
expected 4
actual 4
idx 1609
expected 4
actual 4
idx 1611
expected 3
actual 3
idx 1613
expected 8
actual 8
idx 1614
expected 1
actual 1
idx 1615
expected 7
actual 7
idx 1616
expected 7
actual 7
idx 1619
expected 4
actual 4
idx 1621
expected 1
actual 1
idx 1622
expected 4
actual 4
idx 1624
expected 4
actual 4
idx 1625
expected 3
actual 3
idx 1627
expected 9
actual 9
idx 1628
expected 4
actual 4
idx 1629
expected 9
actual 9
idx 1630
expected 9
actual 9
idx 1631
expected 1
actual 1
idx 1632
expected 8
actual 8
idx 1634
expected 6
actual 6
idx 1635
expected 7
actual 7
idx 1636
expected 5
actual 5
idx 1637
expected 5
actual 5
idx 1639
expected 9
actual 9
idx 1641
expected 6
actual 6
idx 1642
expected 5
actual 5
idx 1643
expected 9
actual 9
idx 1645
expected 4
actual 4
idx 1646
expec

...

idx 7934
expected 8
actual 8
idx 7935
expected 7
actual 7
idx 7936
expected 1
actual 1
idx 7937
expected 1
actual 1
idx 7938
expected 2
actual 2
idx 7940
expected 2
actual 2
idx 7943
expected 1
actual 1
idx 7944
expected 2
actual 2
idx 7945
expected 6
actual 6
idx 7946
expected 1
actual 1
idx 7948
expected 1
actual 1
idx 7949
expected 8
actual 8
idx 7950
expected 9
actual 9
idx 7951
expected 2
actual 2
idx 7952
expected 3
actual 3
idx 7953
expected 1
actual 1
idx 7954
expected 9
actual 9
idx 7955
expected 9
actual 9
idx 7957
expected 4
actual 4
idx 7958
expected 5
actual 5
idx 7960
expected 0
actual 0
idx 7964
expected 1
actual 1
idx 7965
expected 7
actual 7
idx 7966
expected 7
actual 7
idx 7968
expected 8
actual 8
idx 7969
expected 6
actual 6
idx 7971
expected 5
actual 5
idx 7972
expected 5
actual 5
idx 7973
expected 1
actual 1
idx 7974
expected 6
actual 6
idx 7977
expected 8
actual 8
idx 7978
expected 4
actual 4
idx 7982
expected 9
actual 9
idx 7984
expected 8
actual 8
idx 7985
expec

expected 8
actual 8
idx 8516
expected 8
actual 8
idx 8517
expected 1
actual 1
idx 8519
expected 7
actual 7
idx 8520
expected 8
actual 8
idx 8522
expected 1
actual 1
idx 8523
expected 6
actual 6
idx 8527
expected 1
actual 1
idx 8528
expected 3
actual 3
idx 8530
expected 8
actual 8
idx 8531
expected 8
actual 8
idx 8533
expected 6
actual 6
idx 8534
expected 6
actual 6
idx 8535
expected 7
actual 7
idx 8537
expected 0
actual 0
idx 8538
expected 7
actual 7
idx 8540
expected 4
actual 4
idx 8541
expected 8
actual 8
idx 8543
expected 0
actual 0
idx 8544
expected 6
actual 6
idx 8546
expected 8
actual 8
idx 8547
expected 6
actual 6
idx 8550
expected 1
actual 1
idx 8552
expected 6
actual 6
idx 8554
expected 0
actual 0
idx 8555
expected 5
actual 5
idx 8556
expected 4
actual 4
idx 8557
expected 3
actual 3
idx 8558
expected 2
actual 2
idx 8559
expected 1
actual 1
idx 8560
expected 7
actual 7
idx 8561
expected 8
actual 8
idx 8564
expected 1
actual 1
idx 8565
expected 7
actual 7
idx 8567
expected 8
act

idx 9016
expected 1
actual 1
idx 9017
expected 0
actual 0
idx 9018
expected 8
actual 8
idx 9019
expected 9
actual 9
idx 9020
expected 7
actual 7
idx 9021
expected 0
actual 0
idx 9022
expected 3
actual 3
idx 9023
expected 8
actual 8
idx 9024
expected 0
actual 0
idx 9025
expected 5
actual 5
idx 9026
expected 9
actual 9
idx 9027
expected 5
actual 5
idx 9028
expected 8
actual 8
idx 9029
expected 4
actual 4
idx 9031
expected 0
actual 0
idx 9032
expected 9
actual 9
idx 9036
expected 4
actual 4
idx 9037
expected 9
actual 9
idx 9038
expected 2
actual 2
idx 9041
expected 5
actual 5
idx 9042
expected 1
actual 1
idx 9043
expected 3
actual 3
idx 9044
expected 2
actual 2
idx 9045
expected 0
actual 0
idx 9046
expected 0
actual 0
idx 9047
expected 4
actual 4
idx 9048
expected 0
actual 0
idx 9049
expected 6
actual 6
idx 9050
expected 5
actual 5
idx 9051
expected 8
actual 8
idx 9052
expected 0
actual 0
idx 9053
expected 5
actual 5
idx 9054
expected 8
actual 8
idx 9055
expected 6
actual 6
idx 9057
expec

expected 8
actual 8
idx 9399
expected 7
actual 7
idx 9400
expected 6
actual 6
idx 9401
expected 9
actual 9
idx 9402
expected 6
actual 6
idx 9405
expected 8
actual 8
idx 9406
expected 5
actual 5
idx 9407
expected 6
actual 6
idx 9408
expected 6
actual 6
idx 9409
expected 9
actual 9
idx 9410
expected 5
actual 5
idx 9411
expected 7
actual 7
idx 9412
expected 8
actual 8
idx 9413
expected 0
actual 0
idx 9415
expected 0
actual 0
idx 9416
expected 7
actual 7
idx 9417
expected 4
actual 4
idx 9418
expected 8
actual 8
idx 9419
expected 2
actual 2
idx 9420
expected 5
actual 5
idx 9421
expected 1
actual 1
idx 9422
expected 3
actual 3
idx 9424
expected 2
actual 2
idx 9425
expected 6
actual 6
idx 9427
expected 1
actual 1
idx 9429
expected 4
actual 4
idx 9432
expected 1
actual 1
idx 9434
expected 7
actual 7
idx 9438
expected 7
actual 7
idx 9439
expected 0
actual 0
idx 9440
expected 8
actual 8
idx 9441
expected 4
actual 4
idx 9442
expected 4
actual 4
idx 9443
expected 5
actual 5
idx 9444
expected 7
act

idx 9795
expected 4
actual 4
idx 9796
expected 9
actual 9
idx 9797
expected 1
actual 1
idx 9798
expected 5
actual 5
idx 9800
expected 1
actual 1
idx 9802
expected 0
actual 0
idx 9803
expected 1
actual 1
idx 9804
expected 5
actual 5
idx 9806
expected 8
actual 8
idx 9807
expected 1
actual 1
idx 9811
expected 1
actual 1
idx 9813
expected 1
actual 1
idx 9814
expected 8
actual 8
idx 9815
expected 5
actual 5
idx 9816
expected 1
actual 1
idx 9817
expected 9
actual 9
idx 9821
expected 4
actual 4
idx 9823
expected 5
actual 5
idx 9824
expected 1
actual 1
idx 9826
expected 0
actual 0
idx 9827
expected 4
actual 4
idx 9828
expected 9
actual 9
idx 9829
expected 5
actual 5
idx 9830
expected 1
actual 1
idx 9831
expected 2
actual 2
idx 9834
expected 8
actual 8
idx 9835
expected 7
actual 7
idx 9836
expected 2
actual 2
idx 9837
expected 4
actual 4
idx 9838
expected 5
actual 5
idx 9839
expected 3
actual 3
idx 9841
expected 0
actual 0
idx 9843
expected 1
actual 1
idx 9845
expected 9
actual 9
idx 9846
expec

In [31]:
print('wrong_idx', len(wrong_idx))
print('right_idx', len(right_idx))

wrong_idx 2350
right_idx 7650


In [74]:
label_count = 0
for idx in wrong_idx:
    actual_label = get_int_label(y_test[idx])
        
    x_test_neuron_cluster = get_neuron_cluster_with_one_data(x_test[idx])
    x_test_sim, x_test_label = get_max_class_label(x_test_neuron_cluster)
    
    if actual_label != x_test_label:
        label_count = label_count + 1
        print('actual label', actual_label )
        print('expected label', x_test_label)
        #print('similarity', x_test_sim)
        
print('label_count', label_count)

nth= 4144
nth= 4003
actual label 5
expected label 4
nth= 4124
actual label 2
expected label 9
nth= 4168
actual label 2
expected label 3
nth= 4208
actual label 5
expected label 7
nth= 4244
actual label 9
expected label 7
nth= 3929
actual label 0
expected label 2
nth= 4344
actual label 3
expected label 6
nth= 4243
nth= 4222
actual label 7
expected label 3
nth= 3959
actual label 4
expected label 3
nth= 4114
nth= 3976
actual label 3
expected label 5
nth= 4233
nth= 3916
actual label 2
expected label 0
nth= 4227
actual label 3
expected label 5
nth= 4223
nth= 3746
nth= 4105
actual label 0
expected label 9
nth= 4173
actual label 3
expected label 5
nth= 4046
nth= 3764
nth= 4070
actual label 2
expected label 5
nth= 4037
actual label 3
expected label 6
nth= 4125
actual label 6
expected label 4
nth= 3882
nth= 3985
nth= 4135
actual label 1
expected label 6
nth= 3870
nth= 3701
actual label 9
expected label 0
nth= 4222
actual label 3
expected label 5
nth= 4183
actual label 2
expected label 6
nth= 404

nth= 4040
nth= 4003
actual label 4
expected label 2
nth= 3917
actual label 3
expected label 4
nth= 4351
actual label 4
expected label 7
nth= 4030
actual label 5
expected label 2
nth= 4054
actual label 3
expected label 5
nth= 3874
actual label 2
expected label 0
nth= 3917
actual label 3
expected label 5
nth= 4087
actual label 7
expected label 4
nth= 4303
actual label 1
expected label 9
nth= 4111
actual label 2
expected label 6
nth= 4136
actual label 5
expected label 3
nth= 4214
actual label 4
expected label 7
nth= 3971
actual label 2
expected label 4
nth= 4054
actual label 6
expected label 9
nth= 4060
actual label 2
expected label 7
nth= 4009
actual label 2
expected label 6
nth= 4083
actual label 7
expected label 1
nth= 3964
actual label 3
expected label 5
nth= 4073
nth= 3923
nth= 4357
actual label 9
expected label 1
nth= 3982
actual label 3
expected label 5
nth= 4299
actual label 3
expected label 6
nth= 4057
actual label 1
expected label 9
nth= 4227
actual label 1
expected label 4
nth=

nth= 4055
actual label 2
expected label 6
nth= 4068
actual label 2
expected label 5
nth= 3983
actual label 7
expected label 4
nth= 3953
nth= 4120
actual label 2
expected label 6
nth= 4118
actual label 2
expected label 5
nth= 3952
actual label 0
expected label 4
nth= 4338
actual label 2
expected label 9
nth= 4382
nth= 4097
actual label 4
expected label 5
nth= 4220
actual label 9
expected label 1
nth= 4336
nth= 4090
actual label 2
expected label 5
nth= 4163
actual label 4
expected label 6
nth= 4088
actual label 7
expected label 9
nth= 4135
nth= 4213
actual label 0
expected label 6
nth= 3992
actual label 7
expected label 4
nth= 4155
nth= 4156
actual label 3
expected label 5
nth= 3879
actual label 5
expected label 3
nth= 3914
actual label 2
expected label 0
nth= 3819
actual label 0
expected label 2
nth= 4000
actual label 2
expected label 5
nth= 4042
actual label 5
expected label 4
nth= 3969
actual label 3
expected label 6
nth= 4010
actual label 3
expected label 5
nth= 3937
actual label 3
e

nth= 3998
actual label 6
expected label 3
nth= 3562
actual label 2
expected label 0
nth= 4065
nth= 4202
actual label 3
expected label 6
nth= 4120
actual label 5
expected label 2
nth= 4190
actual label 7
expected label 5
nth= 4005
actual label 2
expected label 9
nth= 4036
actual label 2
expected label 6
nth= 4164
actual label 0
expected label 8
nth= 4099
actual label 6
expected label 3
nth= 4148
actual label 2
expected label 5
nth= 4088
nth= 4100
actual label 0
expected label 1
nth= 4166
actual label 2
expected label 6
nth= 4220
nth= 4139
actual label 9
expected label 7
nth= 4283
actual label 2
expected label 5
nth= 4168
actual label 7
expected label 9
nth= 4085
actual label 3
expected label 5
nth= 3970
actual label 5
expected label 4
nth= 3752
actual label 3
expected label 5
nth= 4207
nth= 3880
actual label 5
expected label 3
nth= 4178
actual label 0
expected label 6
nth= 4102
actual label 9
expected label 5
nth= 4311
actual label 0
expected label 9
nth= 4169
nth= 4282
nth= 3788
actual

nth= 4023
actual label 5
expected label 4
nth= 3986
actual label 5
expected label 7
nth= 4110
nth= 4241
nth= 4310
actual label 5
expected label 1
nth= 4134
actual label 2
expected label 6
nth= 4123
nth= 4097
actual label 9
expected label 0
nth= 3965
actual label 3
expected label 2
nth= 4040
actual label 8
expected label 0
nth= 3868
nth= 3921
nth= 4191
actual label 3
expected label 5
nth= 3883
actual label 0
expected label 2
nth= 3710
actual label 4
expected label 0
nth= 3863
nth= 3818
actual label 5
expected label 2
nth= 3857
nth= 4168
actual label 2
expected label 7
nth= 4063
actual label 5
expected label 7
nth= 4064
actual label 8
expected label 1
nth= 4180
nth= 3953
actual label 2
expected label 4
nth= 4168
actual label 7
expected label 9
nth= 3730
nth= 3842
nth= 4296
nth= 3962
actual label 3
expected label 4
nth= 3944
actual label 8
expected label 0
nth= 4178
nth= 3699
actual label 2
expected label 0
nth= 4127
actual label 0
expected label 1
nth= 4187
actual label 7
expected label 

nth= 3852
actual label 2
expected label 0
nth= 4233
actual label 7
expected label 5
nth= 4260
actual label 2
expected label 6
nth= 4030
actual label 2
expected label 9
nth= 4177
actual label 3
expected label 0
nth= 4256
actual label 3
expected label 5
nth= 4160
actual label 2
expected label 4
nth= 3755
nth= 3934
nth= 4100
actual label 0
expected label 9
nth= 4214
nth= 3991
actual label 7
expected label 5
nth= 3879
actual label 4
expected label 2
nth= 4053
actual label 3
expected label 4
nth= 4198
actual label 4
expected label 5
nth= 3811
nth= 3954
nth= 4103
nth= 4036
nth= 3921
actual label 0
expected label 8
nth= 4311
nth= 4252
actual label 7
expected label 9
nth= 3907
actual label 1
expected label 0
nth= 3975
nth= 4093
actual label 2
expected label 8
nth= 3985
actual label 0
expected label 8
nth= 4112
actual label 3
expected label 4
nth= 3979
actual label 8
expected label 7
nth= 4022
actual label 2
expected label 4
nth= 4040
actual label 0
expected label 8
nth= 4018
actual label 2
exp

actual label 4
expected label 7
nth= 4172
actual label 2
expected label 6
nth= 3965
actual label 7
expected label 9
nth= 3938
nth= 4143
actual label 6
expected label 3
nth= 3983
actual label 2
expected label 6
nth= 3952
actual label 4
expected label 2
nth= 4041
actual label 1
expected label 9
nth= 3927
nth= 3836
actual label 5
expected label 2
nth= 4058
actual label 6
expected label 4
nth= 4141
actual label 2
expected label 6
nth= 3964
actual label 2
expected label 5
nth= 4079
actual label 8
expected label 4
nth= 4042
actual label 3
expected label 5
nth= 4202
actual label 3
expected label 7
nth= 4309
actual label 3
expected label 7
nth= 4106
nth= 4100
nth= 4170
nth= 4206
actual label 5
expected label 9
nth= 4060
actual label 7
expected label 5
nth= 4225
actual label 3
expected label 6
nth= 3992
nth= 4141
actual label 2
expected label 6
nth= 4172
actual label 3
expected label 4
nth= 3893
nth= 4086
nth= 4141
actual label 7
expected label 4
nth= 3949
nth= 4301
actual label 2
expected labe

nth= 4181
nth= 4240
actual label 3
expected label 7
nth= 4007
actual label 5
expected label 4
nth= 4159
actual label 2
expected label 5
nth= 3967
actual label 5
expected label 2
nth= 4069
actual label 3
expected label 5
nth= 4213
actual label 3
expected label 5
nth= 4021
nth= 4123
actual label 7
expected label 6
nth= 3836
actual label 4
expected label 2
nth= 4198
nth= 3885
actual label 8
expected label 0
nth= 4170
actual label 3
expected label 6
nth= 4231
actual label 1
expected label 9
nth= 3836
actual label 3
expected label 4
nth= 4019
actual label 2
expected label 4
nth= 4136
actual label 5
expected label 3
nth= 3965
actual label 3
expected label 5
nth= 4151
nth= 4256
actual label 7
expected label 9
nth= 4120
actual label 7
expected label 3
nth= 4220
actual label 9
expected label 1
nth= 4324
actual label 5
expected label 9
nth= 4102
actual label 0
expected label 6
nth= 4121
actual label 0
expected label 7
nth= 4213
actual label 1
expected label 9
nth= 4030
actual label 7
expected la

nth= 4033
actual label 7
expected label 5
nth= 4126
actual label 5
expected label 3
nth= 4153
actual label 6
expected label 9
nth= 4108
actual label 1
expected label 0
nth= 4187
actual label 6
expected label 3
nth= 4106
nth= 4031
actual label 2
expected label 6
nth= 4145
actual label 3
expected label 5
nth= 4159
actual label 7
expected label 9
nth= 4172
nth= 4120
actual label 4
expected label 6
nth= 4039
nth= 3936
nth= 4133
actual label 2
expected label 3
nth= 4076
nth= 4068
nth= 4168
nth= 3977
actual label 3
expected label 5
nth= 4077
nth= 4220
nth= 4001
actual label 6
expected label 2
nth= 3993
actual label 2
expected label 4
nth= 4244
nth= 3880
actual label 0
expected label 8
nth= 4081
nth= 4094
actual label 3
expected label 5
nth= 4036
actual label 6
expected label 2
nth= 4055
actual label 5
expected label 3
nth= 3988
actual label 8
expected label 0
nth= 4147
actual label 3
expected label 6
nth= 4080
nth= 4125
actual label 2
expected label 4
nth= 4005
actual label 3
expected label 

actual label 5
expected label 4
nth= 4058
actual label 0
expected label 4
nth= 3886
actual label 8
expected label 0
nth= 4087
actual label 3
expected label 7
nth= 4171
nth= 3955
actual label 7
expected label 5
nth= 4136
actual label 3
expected label 4
nth= 3922
actual label 5
expected label 3
nth= 3956
nth= 4145
actual label 3
expected label 5
nth= 4095
actual label 3
expected label 5
nth= 4187
nth= 4055
actual label 7
expected label 9
nth= 4160
actual label 3
expected label 5
nth= 4096
actual label 3
expected label 6
nth= 4019
nth= 4145
actual label 9
expected label 1
nth= 3908
actual label 6
expected label 0
nth= 4175
actual label 5
expected label 9
nth= 4116
actual label 8
expected label 1
nth= 4111
actual label 5
expected label 7
nth= 3958
actual label 2
expected label 5
nth= 4165
actual label 5
expected label 4
nth= 4181
actual label 8
expected label 5
nth= 4110
actual label 3
expected label 5
nth= 4056
actual label 5
expected label 7
nth= 4230
actual label 7
expected label 6
nth=

In [75]:
label_count = 0
for idx in right_idx:
    actual_label = get_int_label(y_test[idx])
        
    x_test_neuron_cluster = get_neuron_cluster_with_one_data(x_test[idx])
    x_test_sim, x_test_label = get_max_class_label(x_test_neuron_cluster)
    
    if actual_label == x_test_label:
        label_count = label_count + 1
    else:
        print('\tactual label', actual_label )
        print('\texpected label', x_test_label)
        #print('\tsimilarity', x_test_sim)
        
print('label_count', label_count)

nth= 4214
nth= 4180
	actual label 8
	expected label 1
nth= 4071
	actual label 8
	expected label 1
nth= 4150
nth= 4111
nth= 4027
nth= 4248
nth= 4051
nth= 3996
nth= 4187
nth= 4006
nth= 4337
nth= 4198
nth= 4185
nth= 4349
nth= 3989
nth= 4179
nth= 4232
	actual label 7
	expected label 9
nth= 4100
nth= 4093
nth= 3772
nth= 3816
	actual label 4
	expected label 0
nth= 4193
nth= 3964
nth= 4135
nth= 4365
nth= 4183
nth= 4147
nth= 4086
nth= 3904
nth= 3949
nth= 4113
nth= 4094
nth= 4123
nth= 4266
nth= 4166
nth= 4139
	actual label 4
	expected label 0
nth= 4172
nth= 4138
nth= 3883
nth= 4268
nth= 3925
nth= 4075
nth= 3992
	actual label 6
	expected label 2
nth= 4113
nth= 4034
nth= 3893
nth= 3769
nth= 3925
nth= 4005
	actual label 3
	expected label 1
nth= 4207
nth= 4149
nth= 4274
nth= 4215
nth= 3706
nth= 3923
nth= 3788
nth= 4153
nth= 4185
	actual label 3
	expected label 1
nth= 3919
nth= 3819
nth= 4208
nth= 4043
nth= 4008
nth= 3758
	actual label 7
	expected label 0
nth= 4127
nth= 4162
nth= 3831
nth= 4059
nth=

nth= 4172
nth= 3880
nth= 4249
nth= 4016
nth= 4148
	actual label 5
	expected label 9
nth= 4159
nth= 4151
nth= 3735
nth= 4109
nth= 3935
nth= 4311
nth= 3958
	actual label 0
	expected label 9
nth= 4065
nth= 3982
nth= 4205
nth= 3995
nth= 4007
nth= 3898
nth= 4115
nth= 4240
nth= 4089
nth= 3982
nth= 3636
nth= 4237
nth= 3778
nth= 3751
nth= 4184
nth= 4276
nth= 4180
nth= 4067
nth= 4352
nth= 3779
nth= 3925
	actual label 7
	expected label 4
nth= 3952
	actual label 4
	expected label 0
nth= 4267
nth= 3863
nth= 4153
nth= 3871
nth= 4117
nth= 4144
nth= 4183
nth= 3952
nth= 4173
nth= 4238
nth= 4094
nth= 4141
nth= 4063
nth= 4053
nth= 3968
nth= 4040
nth= 4045
nth= 3997
	actual label 7
	expected label 5
nth= 4106
nth= 4235
nth= 4186
nth= 4144
nth= 4189
nth= 3932
nth= 4036
nth= 4176
nth= 4166
nth= 4025
nth= 3830
nth= 4051
	actual label 0
	expected label 1
nth= 3922
nth= 4306
nth= 4021
	actual label 5
	expected label 6
nth= 4153
nth= 4034
nth= 3883
nth= 3905
nth= 4295
nth= 4081
nth= 3910
nth= 4276
nth= 4268
nt

nth= 3971
nth= 4192
nth= 3862
nth= 3906
nth= 3742
nth= 4006
nth= 4189
nth= 3979
nth= 4048
nth= 4240
nth= 4116
nth= 4228
nth= 4254
nth= 3997
nth= 3800
nth= 4072
nth= 3956
nth= 4114
nth= 3677
nth= 4172
nth= 4173
	actual label 5
	expected label 3
nth= 4081
nth= 4298
nth= 4150
nth= 4073
nth= 4148
nth= 4039
nth= 4028
nth= 4162
nth= 4172
nth= 4250
nth= 4113
nth= 3932
nth= 4185
nth= 4237
nth= 3745
nth= 4268
nth= 4041
nth= 3933
nth= 4019
nth= 4196
nth= 3974
nth= 4242
nth= 4050
nth= 4120
	actual label 3
	expected label 4
nth= 4193
	actual label 0
	expected label 4
nth= 3889
nth= 4040
nth= 4012
nth= 4114
	actual label 8
	expected label 1
nth= 4191
nth= 4001
nth= 4055
	actual label 3
	expected label 9
nth= 3986
nth= 4058
nth= 4126
	actual label 4
	expected label 5
nth= 4172
nth= 4103
nth= 4227
nth= 3942
nth= 4099
nth= 4018
nth= 4025
nth= 3937
nth= 4034
nth= 4161
nth= 4076
nth= 4055
	actual label 4
	expected label 7
nth= 4138
nth= 4163
nth= 4167
nth= 3953
	actual label 8
	expected label 0
nth= 403

nth= 4117
nth= 4408
nth= 4084
nth= 4248
	actual label 1
	expected label 9
nth= 3928
	actual label 6
	expected label 3
nth= 3997
nth= 4395
	actual label 1
	expected label 7
nth= 4200
nth= 3737
nth= 3812
nth= 3956
nth= 4082
nth= 4260
nth= 4012
nth= 4042
	actual label 0
	expected label 4
nth= 4108
nth= 4047
nth= 3856
nth= 3959
	actual label 4
	expected label 2
nth= 4150
nth= 3857
	actual label 0
	expected label 8
nth= 4187
nth= 4189
nth= 4177
nth= 4146
nth= 4089
nth= 3784
nth= 4089
	actual label 4
	expected label 3
nth= 4059
nth= 4072
	actual label 8
	expected label 6
nth= 4003
nth= 4201
	actual label 0
	expected label 9
nth= 4133
nth= 4270
nth= 3937
nth= 4084
nth= 3817
nth= 3940
	actual label 9
	expected label 0
nth= 4393
nth= 3910
nth= 4052
nth= 3842
nth= 4309
nth= 4127
nth= 3998
nth= 4000
	actual label 4
	expected label 3
nth= 3672
nth= 4189
nth= 4051
nth= 3806
nth= 4035
nth= 4123
nth= 4128
nth= 4222
nth= 3798
nth= 4193
nth= 3905
nth= 3902
nth= 4311
nth= 3536
nth= 4118
nth= 4066
	actua

nth= 4258
	actual label 9
	expected label 6
nth= 4220
nth= 4155
nth= 4263
nth= 4024
nth= 4166
nth= 3716
nth= 4017
nth= 4147
nth= 3926
nth= 4030
nth= 3948
nth= 4204
nth= 4084
nth= 3868
nth= 3959
	actual label 0
	expected label 1
nth= 3801
nth= 3941
	actual label 5
	expected label 3
nth= 4067
nth= 4133
nth= 4030
nth= 3986
nth= 3902
nth= 4106
nth= 4061
nth= 4062
nth= 3846
nth= 4102
nth= 3937
nth= 4243
nth= 4231
nth= 4213
nth= 4286
nth= 4170
	actual label 4
	expected label 5
nth= 4047
nth= 4261
	actual label 0
	expected label 9
nth= 4028
nth= 4076
	actual label 4
	expected label 7
nth= 4055
nth= 4207
nth= 4381
nth= 3859
	actual label 8
	expected label 0
nth= 4113
nth= 4138
nth= 4199
nth= 4000
nth= 4022
nth= 4047
nth= 4010
nth= 4143
nth= 4396
nth= 4303
nth= 4202
	actual label 4
	expected label 6
nth= 3960
nth= 4168
	actual label 4
	expected label 3
nth= 3929
nth= 4289
nth= 4108
nth= 3886
	actual label 1
	expected label 2
nth= 3678
nth= 4015
nth= 3952
nth= 4052
nth= 4261
nth= 4010
nth= 4079


...

nth= 4255
nth= 4230
nth= 3724
	actual label 0
	expected label 8
nth= 3889
nth= 3929
nth= 4055
nth= 4032
nth= 4218
nth= 4206
nth= 3947
nth= 3850
nth= 4160
	actual label 4
	expected label 7
nth= 4103
nth= 4178
nth= 4092
nth= 4175
nth= 4012
nth= 4337
nth= 4073
nth= 4081
	actual label 6
	expected label 1
nth= 4166
nth= 4155
nth= 4040
nth= 4180
nth= 3937
nth= 4060
	actual label 8
	expected label 1
nth= 4012
	actual label 4
	expected label 7
nth= 3895
nth= 4158
nth= 3971
nth= 4010
nth= 4138
nth= 4182
nth= 4107
nth= 3628
nth= 4079
nth= 4013
	actual label 0
	expected label 2
nth= 3919
nth= 3956
nth= 4145
nth= 3784
nth= 4088
nth= 4103
nth= 3851
nth= 3910
nth= 4055
nth= 4310
	actual label 0
	expected label 9
nth= 3818
nth= 4009
nth= 4195
nth= 3979
nth= 4059
nth= 3994
nth= 3767
nth= 4016
nth= 4003
nth= 4185
nth= 4157
nth= 3642
nth= 4274
	actual label 7
	expected label 9
nth= 4215
nth= 4081
nth= 4243
	actual label 1
	expected label 9
nth= 4016
nth= 4094
nth= 4219
nth= 3819
nth= 3866
nth= 4426
nth=

nth= 4132
nth= 3960
nth= 4321
nth= 3918
nth= 4363
nth= 4175
nth= 3779
nth= 4172
nth= 4154
nth= 4122
nth= 4030
nth= 4140
nth= 4166
nth= 3816
nth= 4123
nth= 4058
nth= 3947
nth= 3853
nth= 4019
nth= 3994
nth= 4033
nth= 4294
nth= 4237
	actual label 9
	expected label 3
nth= 4134
nth= 3703
nth= 3755
nth= 4124
nth= 3983
nth= 3884
nth= 3988
nth= 4120
nth= 4319
nth= 3955
nth= 3985
nth= 4077
nth= 4042
nth= 4240
nth= 4336
nth= 4312
nth= 4045
nth= 4090
nth= 4303
nth= 4190
	actual label 5
	expected label 1
nth= 4077
	actual label 7
	expected label 3
nth= 4178
nth= 4237
nth= 4027
nth= 4049
nth= 4090
nth= 4088
nth= 4131
nth= 3950
nth= 4152
nth= 4084
nth= 3785
nth= 3740
nth= 3920
nth= 3841
nth= 3975
nth= 3947
nth= 4144
nth= 3991
nth= 4067
nth= 4222
nth= 4120
nth= 4143
nth= 4173
nth= 4206
nth= 4260
nth= 4171
nth= 4145
nth= 4148
nth= 4242
nth= 3973
	actual label 0
	expected label 8
nth= 3730
nth= 4256
nth= 4022
nth= 4132
nth= 3860
nth= 3923
nth= 4273
nth= 3773
nth= 4091
nth= 4210
nth= 4243
nth= 4133
nth=

nth= 4127
nth= 3981
nth= 4054
	actual label 0
	expected label 8
nth= 3991
	actual label 8
	expected label 0
nth= 4145
nth= 4205
nth= 4074
nth= 3981
	actual label 6
	expected label 2
nth= 4181
nth= 4272
nth= 4084
	actual label 4
	expected label 7
nth= 4013
nth= 4334
nth= 3934
nth= 3875
nth= 3991
nth= 4159
nth= 3933
nth= 3771
nth= 4036
nth= 4025
nth= 3988
	actual label 7
	expected label 4
nth= 4189
nth= 4144
nth= 4142
	actual label 2
	expected label 9
nth= 3949
nth= 4314
nth= 3883
nth= 4227
nth= 4135
nth= 4214
nth= 4183
nth= 4139
nth= 3734
nth= 3707
	actual label 3
	expected label 2
nth= 4067
nth= 4318
nth= 4223
nth= 4010
nth= 4040
	actual label 5
	expected label 3
nth= 3787
nth= 4080
nth= 4102
nth= 4201
nth= 3923
nth= 4234
	actual label 6
	expected label 1
nth= 4096
nth= 4002
nth= 4141
nth= 3866
	actual label 6
	expected label 2
nth= 3898
nth= 3884
nth= 4175
nth= 4108
nth= 4132
nth= 4026
nth= 3832
nth= 4084
nth= 3898
	actual label 7
	expected label 3
nth= 4154
nth= 4168
nth= 3832
nth= 3

nth= 4159
	actual label 1
	expected label 6
nth= 3890
	actual label 5
	expected label 3
nth= 4022
nth= 4159
nth= 4044
nth= 4135
nth= 3991
nth= 4192
nth= 4172
nth= 4015
	actual label 2
	expected label 6
nth= 3938
nth= 4111
nth= 3819
nth= 4189
nth= 3938
	actual label 6
	expected label 2
nth= 3969
nth= 4077
nth= 3768
nth= 4202
nth= 4139
nth= 4006
nth= 4069
nth= 4000
nth= 4153
nth= 3985
nth= 4076
nth= 3950
nth= 4076
nth= 4064
nth= 4142
nth= 4114
nth= 3691
nth= 4234
nth= 4135
nth= 4322
nth= 4150
nth= 3774
nth= 4216
nth= 4350
nth= 4153
nth= 4306
	actual label 6
	expected label 3
nth= 3718
nth= 4060
	actual label 2
	expected label 5
nth= 4251
nth= 3650
nth= 4003
nth= 4298
nth= 4136
nth= 4120
nth= 3919
nth= 3727
nth= 4019
nth= 4238
nth= 3994
nth= 4173
	actual label 0
	expected label 1
nth= 3758
nth= 4248
	actual label 0
	expected label 1
nth= 3705
nth= 4072
nth= 4063
nth= 4252
nth= 4261
nth= 3757
nth= 3890
nth= 4108
nth= 3828
nth= 4192
nth= 4142
nth= 4083
nth= 4238
nth= 3827
nth= 4189
nth= 406

nth= 4228
nth= 4172
nth= 3835
nth= 4168
nth= 3830
nth= 3931
nth= 4210
nth= 4179
nth= 4218
nth= 3907
nth= 4082
nth= 4199
nth= 4154
nth= 4058
	actual label 3
	expected label 5
nth= 3872
nth= 4183
nth= 4067
nth= 4204
nth= 4049
nth= 3977
	actual label 9
	expected label 0
nth= 4046
nth= 4148
nth= 3919
nth= 4165
nth= 4057
nth= 3697
nth= 3932
nth= 4204
nth= 4195
nth= 4218
nth= 3904
nth= 4196
nth= 4363
nth= 4087
nth= 3674
nth= 4136
nth= 4045
nth= 4031
nth= 4240
nth= 4038
nth= 3614
nth= 4237
nth= 4200
nth= 3988
nth= 4105
	actual label 4
	expected label 2
nth= 3814
nth= 4290
nth= 4259
nth= 4045
	actual label 8
	expected label 9
nth= 4171
nth= 3997
nth= 4077
	actual label 4
	expected label 7
nth= 4204
nth= 4006
nth= 4356
nth= 4265
nth= 4196
nth= 4104
nth= 4138
nth= 3990
nth= 4015
nth= 3791
nth= 4203
nth= 4124
nth= 4217
nth= 3812
nth= 3800
nth= 3835
nth= 3964
nth= 4237
nth= 4112
nth= 4138
nth= 3994
nth= 4081
nth= 4027
nth= 3997
nth= 4052
	actual label 5
	expected label 3
nth= 3913
nth= 3814
nth= 4

##### measuring test effectiveness for original test data

In [20]:
def selectEffectiveTestData(origin_x_data, origin_y_data):
    diff_x_test = []
    diff_y_test = []

    data_size = len(origin_x_data)
    for idx in range(data_size):
        x_test_neuron_cluster = get_neuron_cluster_with_one_data(origin_x_data[idx])
        x_test_sim, x_test_label = get_max_class_label(x_test_neuron_cluster)
        
        actual_label = get_int_label(origin_y_data[idx])
        
        if  x_test_label != actual_label:
            diff_x_test.append(origin_x_data[idx])
            diff_y_test.append(origin_y_data[idx])
#         else:
#             print('x_test_label', x_test_label, 'actual_label', actual_label)
        
    np_diff_x_test = np.array(diff_x_test)
    np_diff_y_test = np.array(diff_y_test)
    
    return np_diff_x_test, np_diff_y_test

In [138]:
index_range = range(0, 10000)

In [148]:
start_t = time.time()

selected_x_test, selected_y_test = selectEffectiveTestData(x_test[index_range], y_test[index_range])

end_t = time.time()
print(str(end_t - start_t) + 'sec')

print(selected_x_test.shape)
print(selected_y_test.shape)

257.4165577888489sec
(9000, 32, 32, 3)
(9000, 10)


In [149]:
sel_scores = model_v1.evaluate(selected_x_test, selected_y_test, verbose=1)
print('Test loss:', sel_scores[0])
print('Test accuracy:', sel_scores[1])

282/282 [==============================] - 3s 11ms/step - loss: 1.4137 - accuracy: 0.7547
Test loss: 1.4136791229248047
Test accuracy: 0.7546666860580444


###### Data Augmentation to multiple image

In [34]:
from PIL import Image
from PIL import ImageFilter
from PIL import ImageEnhance

In [35]:
def getAugmentedImages(img, loop_count):
    images = []
    
    pil_img = Image.fromarray(img)
    for idx in range(loop_count):        
        images.append(np.array(pil_img.rotate(5 * loop_count)))
        images.append(np.array(pil_img.rotate(-5 * loop_count)))
        images.append(np.array(pil_img.transpose(Image.FLIP_LEFT_RIGHT)))
        images.append(np.array(pil_img.transpose(Image.FLIP_LEFT_RIGHT).rotate(5 * loop_count)))
        images.append(np.array(pil_img.transpose(Image.FLIP_LEFT_RIGHT).rotate(-5 * loop_count)))
        images.append(np.array(pil_img.filter(ImageFilter.SMOOTH)))
        images.append(np.array(pil_img.filter(ImageFilter.SHARPEN)))
        images.append(np.array(ImageEnhance.Brightness(pil_img).enhance(1.0 + 0.1 * loop_count)))
        images.append(np.array(ImageEnhance.Brightness(pil_img).enhance(1.0 - 0.1 * loop_count)))
        images.append(np.array(ImageEnhance.Color(pil_img).enhance(0.3 - 0.1 * loop_count)))
    
    return images

In [36]:
def getAugmentedLabels(label, loop_count):
    labels = []
    
    for idx1 in range(loop_count):
        for idx2 in range(10):
            labels.append(label)
    
    return labels

##### measuring model accuracy for augmented test data

In [37]:
aug_x_test = []
aug_y_test = []

for idx in range(10000):
    aug_x_test.extend(getAugmentedImages(x_test[idx], 1))
    aug_y_test.extend(getAugmentedLabels(y_test[idx], 1))
    
np_aug_x_test = np.array(aug_x_test)
np_aug_y_test = np.array(aug_y_test)

print(np_aug_x_test.shape)
print(np_aug_y_test.shape)

(100000, 32, 32, 3)
(100000, 10)


In [79]:
aug_scores = model_v1.evaluate(np_aug_x_test, np_aug_y_test, verbose=1)
print('Test loss:', aug_scores[0])
print('Test accuracy:', aug_scores[1])

3125/3125 [==============================] - 59s 19ms/step - loss: 1.5830 - accuracy: 0.7186
Test loss: 1.5830440521240234
Test accuracy: 0.718559980392456


##### measuring test effectiveness for augmented test data

In [69]:
start_t = time.time()

# 약 1시간정도 소요됨
np_diff_x_test, np_diff_y_test = selectEffectiveTestData(np_aug_x_test, np_aug_y_test)

nth= 7163
nth= 6847
nth= 7080
nth= 6966
nth= 6900
nth= 6699
nth= 7180
nth= 7048
nth= 7025
nth= 7088
nth= 7224
nth= 7018
nth= 6772
nth= 6953
nth= 7019
nth= 6784
nth= 7014
nth= 6954
nth= 6977
nth= 6940
nth= 7099
nth= 6914
nth= 7064
nth= 7050
nth= 7149
nth= 6415
nth= 6927
nth= 6825
nth= 6777
nth= 6869
nth= 6897
nth= 6956
nth= 6729
nth= 6877
nth= 6962
nth= 6503
nth= 7065
nth= 6872
nth= 6925
nth= 6872
nth= 6854
nth= 6876
nth= 6883
nth= 6863
nth= 7010
nth= 6702
nth= 7032
nth= 6843
nth= 6846
nth= 6897
nth= 6882
nth= 6838
nth= 6801
nth= 6905
nth= 6912
nth= 6508
nth= 6932
nth= 6719
nth= 6733
nth= 6797
nth= 7090
nth= 7069
nth= 6951
nth= 7165
nth= 7098
nth= 6939
nth= 7167
nth= 7074
nth= 7076
nth= 7069
nth= 6900
nth= 6804
nth= 6898
nth= 6894
nth= 6913
nth= 6448
nth= 6864
nth= 6757
nth= 6718
nth= 6685
nth= 6802
nth= 6806
nth= 6702
nth= 6966
nth= 6951
nth= 6432
nth= 6803
nth= 6652
nth= 6673
nth= 6617
nth= 7028
nth= 7196
nth= 7152
nth= 7212
nth= 7092
nth= 6707
nth= 7124
nth= 7007
nth= 6973
nth= 7022


nth= 6813
nth= 6706
nth= 6734
nth= 6711
nth= 6884
nth= 6957
nth= 6675
nth= 6754
nth= 6863
nth= 6453
nth= 6935
nth= 6771
nth= 6762
nth= 6744
nth= 6591
nth= 6629
nth= 6102
nth= 6634
nth= 6416
nth= 6026
nth= 6387
nth= 6271
nth= 6284
nth= 6243
nth= 6802
nth= 7015
nth= 6551
nth= 6758
nth= 6758
nth= 6445
nth= 6918
nth= 6659
nth= 6722
nth= 6738
nth= 6889
nth= 6932
nth= 6869
nth= 6875
nth= 6884
nth= 6653
nth= 6791
nth= 6763
nth= 6802
nth= 6688
nth= 6591
nth= 6637
nth= 6312
nth= 6681
nth= 6576
nth= 6068
nth= 6389
nth= 6267
nth= 6213
nth= 6239
nth= 6958
nth= 7045
nth= 6890
nth= 6998
nth= 7059
nth= 6552
nth= 7034
nth= 6860
nth= 6874
nth= 6903
nth= 7026
nth= 7075
nth= 7047
nth= 7136
nth= 6977
nth= 6641
nth= 7064
nth= 6951
nth= 6917
nth= 6993
nth= 6805
nth= 6724
nth= 6357
nth= 6792
nth= 6852
nth= 6146
nth= 6521
nth= 6367
nth= 6360
nth= 6411
nth= 6974
nth= 6754
nth= 6832
nth= 6848
nth= 6893
nth= 6558
nth= 6910
nth= 6766
nth= 6677
nth= 6807
nth= 7127
nth= 6979
nth= 6913
nth= 6911
nth= 7079
nth= 6388


nth= 6751
nth= 6606
nth= 6684
nth= 6759
nth= 6363
nth= 6909
nth= 6693
nth= 6591
nth= 6650
nth= 6779
nth= 6528
nth= 6456
nth= 6634
nth= 6828
nth= 5939
nth= 6654
nth= 6371
nth= 6330
nth= 6323
nth= 6870
nth= 6818
nth= 6898
nth= 6731
nth= 6808
nth= 6393
nth= 7004
nth= 6854
nth= 6782
nth= 6750
nth= 6930
nth= 6942
nth= 6681
nth= 6885
nth= 6747
nth= 6583
nth= 6703
nth= 6663
nth= 6665
nth= 6697
nth= 6609
nth= 6829
nth= 6548
nth= 6748
nth= 6732
nth= 6188
nth= 6740
nth= 6426
nth= 6422
nth= 6468
nth= 7044
nth= 7094
nth= 7043
nth= 7084
nth= 6880
nth= 6991
nth= 7034
nth= 6989
nth= 7049
nth= 7129
nth= 7018
nth= 7009
nth= 6873
nth= 7141
nth= 6947
nth= 6840
nth= 7111
nth= 6969
nth= 7056
nth= 7083
nth= 7100
nth= 6889
nth= 6821
nth= 6887
nth= 6994
nth= 6330
nth= 6957
nth= 6902
nth= 6855
nth= 6962
nth= 6758
nth= 6681
nth= 6586
nth= 6876
nth= 6854
nth= 6384
nth= 6731
nth= 6656
nth= 6634
nth= 6607
nth= 7120
nth= 7161
nth= 6904
nth= 7140
nth= 7072
nth= 6840
nth= 7051
nth= 7006
nth= 7051
nth= 7110
nth= 7236


nth= 6334
nth= 6927
nth= 6715
nth= 6694
nth= 6744
nth= 6733
nth= 6741
nth= 6943
nth= 6846
nth= 6904
nth= 6370
nth= 6829
nth= 6649
nth= 6705
nth= 6685
nth= 6872
nth= 6911
nth= 7016
nth= 6958
nth= 7082
nth= 6895
nth= 7135
nth= 7083
nth= 7119
nth= 7084
nth= 6914
nth= 6887
nth= 6521
nth= 6751
nth= 6856
nth= 6597
nth= 7040
nth= 6872
nth= 6884
nth= 6880
nth= 7054
nth= 7171
nth= 6679
nth= 6974
nth= 6805
nth= 6894
nth= 7096
nth= 7063
nth= 7063
nth= 7049
nth= 7108
nth= 6992
nth= 7025
nth= 7090
nth= 7202
nth= 6950
nth= 7152
nth= 7081
nth= 7072
nth= 7056
nth= 6964
nth= 7226
nth= 7104
nth= 7093
nth= 7005
nth= 6772
nth= 7021
nth= 6930
nth= 6886
nth= 6912
nth= 6972
nth= 6769
nth= 6738
nth= 6816
nth= 6848
nth= 6448
nth= 6711
nth= 6634
nth= 6567
nth= 6605
nth= 6772
nth= 6672
nth= 6487
nth= 6730
nth= 6818
nth= 6105
nth= 6583
nth= 6365
nth= 6395
nth= 6386
nth= 6927
nth= 6982
nth= 6920
nth= 6997
nth= 7041
nth= 6700
nth= 7053
nth= 7034
nth= 6961
nth= 6924
nth= 6862
nth= 6918
nth= 6497
nth= 6789
nth= 6960


nth= 6864
nth= 6750
nth= 6808
nth= 6843
nth= 6564
nth= 6872
nth= 6734
nth= 6760
nth= 6911
nth= 7091
nth= 7017
nth= 6913
nth= 7030
nth= 6916
nth= 6800
nth= 7067
nth= 6939
nth= 6971
nth= 6961
nth= 6941
nth= 6941
nth= 6806
nth= 6922
nth= 6872
nth= 6654
nth= 6955
nth= 6834
nth= 6808
nth= 6869
nth= 6928
nth= 7169
nth= 7149
nth= 7292
nth= 7227
nth= 6843
nth= 7251
nth= 7162
nth= 7089
nth= 7103
nth= 6984
nth= 6906
nth= 6880
nth= 6979
nth= 6780
nth= 6740
nth= 6900
nth= 6852
nth= 6839
nth= 6905
nth= 7391
nth= 7380
nth= 7402
nth= 7490
nth= 7260
nth= 7107
nth= 7308
nth= 7338
nth= 7333
nth= 7307
nth= 6957
nth= 6971
nth= 7015
nth= 6892
nth= 7051
nth= 6826
nth= 6932
nth= 6932
nth= 6932
nth= 6860
nth= 6867
nth= 6940
nth= 6772
nth= 6867
nth= 6841
nth= 6400
nth= 6884
nth= 6754
nth= 6728
nth= 6757
nth= 6491
nth= 6561
nth= 6203
nth= 6621
nth= 6455
nth= 6028
nth= 6202
nth= 6142
nth= 6128
nth= 6117
nth= 6898
nth= 6973
nth= 6781
nth= 6988
nth= 7013
nth= 6580
nth= 6866
nth= 6720
nth= 6747
nth= 6625
nth= 7026


...

nth= 7004
nth= 7006
nth= 6925
nth= 7087
nth= 7171
nth= 7043
nth= 7133
nth= 7043
nth= 6872
nth= 7171
nth= 7059
nth= 7011
nth= 7074
nth= 6774
nth= 6764
nth= 6991
nth= 6815
nth= 7055
nth= 6702
nth= 7083
nth= 6862
nth= 6839
nth= 6805
nth= 7136
nth= 6928
nth= 6861
nth= 6997
nth= 7110
nth= 6854
nth= 7110
nth= 7029
nth= 7001
nth= 6920
nth= 6461
nth= 6536
nth= 6179
nth= 6525
nth= 6370
nth= 6248
nth= 6501
nth= 6352
nth= 6332
nth= 6352
nth= 6527
nth= 6585
nth= 6090
nth= 6376
nth= 6441
nth= 6048
nth= 6543
nth= 6327
nth= 6285
nth= 6309
nth= 6614
nth= 6730
nth= 6655
nth= 6700
nth= 6703
nth= 6528
nth= 6861
nth= 6679
nth= 6733
nth= 6800
nth= 6662
nth= 6631
nth= 6428
nth= 6693
nth= 6675
nth= 6055
nth= 6552
nth= 6388
nth= 6393
nth= 6372
nth= 6676
nth= 6679
nth= 6816
nth= 6879
nth= 6832
nth= 6301
nth= 6432
nth= 6367
nth= 6457
nth= 6483
nth= 7215
nth= 6894
nth= 6981
nth= 7109
nth= 7098
nth= 6678
nth= 7240
nth= 7186
nth= 7152
nth= 7066
nth= 6800
nth= 6735
nth= 6667
nth= 6964
nth= 6972
nth= 6422
nth= 6586


nth= 7010
nth= 6853
nth= 7239
nth= 7099
nth= 7073
nth= 7033
nth= 7113
nth= 7379
nth= 7083
nth= 7240
nth= 7192
nth= 6792
nth= 7077
nth= 6970
nth= 6983
nth= 6960
nth= 6692
nth= 6762
nth= 6371
nth= 6689
nth= 6829
nth= 6189
nth= 6517
nth= 6346
nth= 6335
nth= 6292
nth= 6961
nth= 7080
nth= 6960
nth= 6970
nth= 6968
nth= 6628
nth= 7073
nth= 6980
nth= 6911
nth= 6894
nth= 6923
nth= 6909
nth= 6775
nth= 6876
nth= 6819
nth= 6560
nth= 6956
nth= 6802
nth= 6750
nth= 6740
nth= 6882
nth= 6967
nth= 7019
nth= 7140
nth= 7153
nth= 6653
nth= 7050
nth= 6912
nth= 6928
nth= 6890
nth= 6941
nth= 6974
nth= 6716
nth= 6871
nth= 6759
nth= 6526
nth= 6792
nth= 6678
nth= 6673
nth= 6725
nth= 6579
nth= 6626
nth= 6262
nth= 6474
nth= 6561
nth= 6284
nth= 6375
nth= 6374
nth= 6383
nth= 6398
nth= 6894
nth= 6819
nth= 6723
nth= 6743
nth= 6722
nth= 6611
nth= 7108
nth= 6937
nth= 6872
nth= 6959
nth= 7250
nth= 7030
nth= 7004
nth= 7202
nth= 7228
nth= 6822
nth= 7120
nth= 7034
nth= 7110
nth= 7074
nth= 6761
nth= 6670
nth= 6559
nth= 6846


nth= 7272
nth= 7231
nth= 7023
nth= 7231
nth= 7204
nth= 6948
nth= 7208
nth= 7147
nth= 7125
nth= 7146
nth= 6190
nth= 6221
nth= 6252
nth= 6524
nth= 6458
nth= 5924
nth= 6360
nth= 6149
nth= 6038
nth= 6199
nth= 6588
nth= 6775
nth= 6447
nth= 6758
nth= 6735
nth= 6370
nth= 6393
nth= 6411
nth= 6407
nth= 6436
nth= 6988
nth= 7077
nth= 6977
nth= 7064
nth= 7108
nth= 6793
nth= 6985
nth= 7020
nth= 7010
nth= 7045
nth= 6467
nth= 6692
nth= 6726
nth= 6912
nth= 6705
nth= 6383
nth= 6816
nth= 6662
nth= 6602
nth= 6579
nth= 6431
nth= 6553
nth= 6201
nth= 6511
nth= 6444
nth= 6318
nth= 6454
nth= 6412
nth= 6433
nth= 6339
nth= 6950
nth= 6829
nth= 6607
nth= 6882
nth= 6846
nth= 6224
nth= 6960
nth= 6618
nth= 6611
nth= 6732
nth= 7150
nth= 7212
nth= 7218
nth= 7325
nth= 7151
nth= 7023
nth= 7277
nth= 7245
nth= 7282
nth= 7242
nth= 7476
nth= 7465
nth= 7591
nth= 7459
nth= 7361
nth= 6926
nth= 7500
nth= 7376
nth= 7402
nth= 7364
nth= 6881
nth= 6914
nth= 6616
nth= 6668
nth= 6798
nth= 6590
nth= 6888
nth= 6742
nth= 6764
nth= 6767


nth= 7043
nth= 7001
nth= 6998
nth= 7065
nth= 6870
nth= 7068
nth= 7051
nth= 7105
nth= 7078
nth= 7036
nth= 6883
nth= 6898
nth= 7005
nth= 6914
nth= 6445
nth= 7122
nth= 6865
nth= 6876
nth= 6873
nth= 7102
nth= 7096
nth= 6825
nth= 6991
nth= 7110
nth= 6693
nth= 7136
nth= 7035
nth= 7101
nth= 7089
nth= 6704
nth= 6716
nth= 6374
nth= 6655
nth= 6794
nth= 6157
nth= 6455
nth= 6395
nth= 6308
nth= 6306
nth= 6877
nth= 6768
nth= 6829
nth= 7029
nth= 6827
nth= 6660
nth= 6915
nth= 6845
nth= 6844
nth= 6916
nth= 7098
nth= 7029
nth= 6924
nth= 7081
nth= 7121
nth= 6773
nth= 7195
nth= 7065
nth= 7063
nth= 7113
nth= 6835
nth= 6847
nth= 6792
nth= 6737
nth= 6774
nth= 6598
nth= 6885
nth= 6813
nth= 6832
nth= 6730
nth= 6427
nth= 6723
nth= 6323
nth= 6497
nth= 6447
nth= 6169
nth= 6365
nth= 6320
nth= 6303
nth= 6332
nth= 7001
nth= 6793
nth= 6671
nth= 6796
nth= 6918
nth= 6531
nth= 6859
nth= 6741
nth= 6771
nth= 6794
nth= 6846
nth= 6748
nth= 6582
nth= 6847
nth= 6876
nth= 6347
nth= 6866
nth= 6729
nth= 6674
nth= 6669
nth= 6404


nth= 7023
nth= 6910
nth= 6862
nth= 6882
nth= 6804
nth= 6892
nth= 6558
nth= 6677
nth= 6927
nth= 6501
nth= 6730
nth= 6696
nth= 6742
nth= 6666
nth= 7061
nth= 6959
nth= 6936
nth= 7029
nth= 7098
nth= 6886
nth= 6990
nth= 7057
nth= 7011
nth= 6973
nth= 6965
nth= 6846
nth= 6775
nth= 7026
nth= 6868
nth= 6649
nth= 6890
nth= 6806
nth= 6738
nth= 6692
nth= 7026
nth= 6897
nth= 6729
nth= 6887
nth= 6706
nth= 6514
nth= 7025
nth= 6732
nth= 6763
nth= 6732
nth= 7004
nth= 6858
nth= 6913
nth= 6871
nth= 6997
nth= 6476
nth= 6801
nth= 6627
nth= 6614
nth= 6691
nth= 6424
nth= 6481
nth= 6173
nth= 6428
nth= 6460
nth= 6033
nth= 6256
nth= 6189
nth= 6172
nth= 6166
nth= 7169
nth= 7032
nth= 6877
nth= 7008
nth= 7041
nth= 6710
nth= 7107
nth= 6967
nth= 6976
nth= 7044
nth= 6872
nth= 6912
nth= 6885
nth= 6758
nth= 6868
nth= 6434
nth= 6888
nth= 6804
nth= 6781
nth= 6699
nth= 6621
nth= 6680
nth= 6432
nth= 6762
nth= 6641
nth= 6248
nth= 6396
nth= 6277
nth= 6347
nth= 6309
nth= 7239
nth= 7286
nth= 7106
nth= 7216
nth= 7262
nth= 6999


In [70]:
end_t = time.time()
print(str(end_t - start_t) + 'sec')

print(np_diff_x_test.shape)
print(np_diff_y_test.shape)

2578.1700077056885sec
(90000, 32, 32, 3)
(90000, 10)


In [71]:
diff_scores = model_v1.evaluate(np_diff_x_test, np_diff_y_test, verbose=1)
print('Test loss:', diff_scores[0])
print('Test accuracy:', diff_scores[1])

2813/2813 [==============================] - 25s 9ms/step - loss: 1.6186 - accuracy: 0.7129
Test loss: 1.6185803413391113
Test accuracy: 0.7128555774688721


In [11]:
import random

In [12]:
random_number = random.sample(range(0, 10), 10)

print(random_number)

[5, 0, 6, 1, 8, 4, 3, 7, 9, 2]


In [8]:
random_numbers = np.random.choice(range(0, 10), size=10, replace=False)

print(random_numbers)

[2 9 0 7 5 3 1 6 8 4]


In [14]:
print(type(random_number))
print(type(random_numbers))

<class 'list'>
<class 'numpy.ndarray'>


In [16]:
for idx in random_number:
    print(idx)

print()

for idx in random_numbers:
    print(idx)

5
0
6
1
8
4
3
7
9
2

2
9
0
7
5
3
1
6
8
4
